<a href="https://colab.research.google.com/github/daver987/unsloth-mojo/blob/main/Unsloth_Mojo_Puzzles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsloth Challenge Mapped to Mojo/Max

We're still hiring on a rolling basis (interns, junior engs, senior)
### Email me daniel at unsloth ai with your resume, Github repo, what you wanna work on, your past experience on projects (uni included). Do apply if you have implemented Llama in PyTorch from scratch :)

### 🦥 Unsloth is growing! Come join us :)
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a>

Up to $500K USD salary + bonus equity, health care benefits + other benefits, USA relocation etc! Complete some puzzles and earn points!

* We encourage you to use AI for coding!<ins> No experience or PhD / Masters needed</ins> - just get enough points for consideration!
* There are <ins>negative points</ins> for incorrect submissions. Read each criteria! Read [Submission](#SUBMISSION) steps.

| Role              | Compensation   | Role Description | Points Needed |
| ----------------- | -------------- | ----------- | --- |
| Founding Engineer | \$400K to \$500K & equity | Help push Unsloth forward - bug fixes, core features, UI, kernels, nearly anything! | 47 |
| ML Engineer | \$250K to \$300K & equity | Help with FSDP2, Float8, Float4, kernels, Unsloth core and more! | 32 |
| ML Intern | up to \$150K py | Implementing specific features in Unsloth core. Can be remote.  | 18 |

1. [Convert `nf4` to Triton](#NF4) [Difficulty: Hard] [Max points: 14]
2. [Make `QLoRA` work with `FSDP2`](#FSDP2) [Difficulty: Medium to Hard] [Max points: 12]
3. [Make `torch.compile` work without graph breaks for QLoRA](#COMPILE) [Difficulty: Easy to Medium] [Max points: 9]
4. [Help solve 🦥 Unsloth issues!](#ISSUES) [Difficulty: Varies] [Max points: 12]
5. [Memory Efficient Backprop](#MATH) [Difficulty: Medium to Hard] [Max points: 10]
6. [Submission steps](#SUBMISSION)

### 🦥 Who are we?
* 1.58bit DeepSeek R1 GGUFs [Tweet](https://x.com/UnslothAI/status/1883899061893546254) and [HF Model Page](https://huggingface.co/unsloth/DeepSeek-R1-GGUF)
* GRPO Llama 3.1 8B on a free Colab [Tweet](https://x.com/UnslothAI/status/1887562753126408210)
* Gemma bug fixes [Tweet](https://x.com/danielhanchen/status/1765446273661075609) and bug fixes for Llama 3, Phi 3, Qwen 2.5 [Details](https://unsloth.ai/blog/phi3) Llama-fying Phi-4 [Details](https://unsloth.ai/blog/phi4)
* Gradient accumulation bug fixes [Tweet](https://x.com/danielhanchen/status/1846235913443262891) 4bit Dynamic Quantization [Details](https://unsloth.ai/blog/dynamic-4bit)
* Unsloth Gradient Checkpointing async offloads activations [Details](https://unsloth.ai/blog/long-context)
* 30K Github Stars [Github](https://github.com/unslothai/unsloth) & 7 million monthly downloads on [Hugging Face](https://huggingface.co/unsloth)
* PyTorch conference [video](https://www.youtube.com/watch?v=PdtKkc5jB4g) AI Engineer World's Fair [video](https://www.youtube.com/watch?v=pRM_P6UfdIc) GPU / CUDA MODE [talk](https://www.youtube.com/watch?v=hfb_AIhDYnA)


### Clarifications:
1. We'll compensate you if we interview you but don't hire you
2. \$100-\$1000 bounties for Task 4
3. Submissions must be Apache-2 licensed
4. Task 4 involves solving Github issues for OSS Unsloth
5. No time limit: rolling basis
6. US based preferred

In [15]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [16]:
# Helpful functions used through the entire notebook
import torch
import torch.nn as nn
from transformers import set_seed
import time
import inspect
import os
major_version, minor_version = torch.cuda.get_device_capability()
HAS_BFLOAT16 = (major_version >= 8)
from inspect import currentframe as _C, getframeinfo
_F = lambda c: getframeinfo(c).lineno # Gets line number
WARN = lambda x: print(f"\033[31m{x}\033[0m") # Red colored warnings

# https://stackoverflow.com/questions/18425225/getting-the-name-of-a-variable-as-a-string
def NAME(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    names = [var_name for var_name, var_val in callers_local_vars if var_val is var]
    return names[0] if len(names) != 0 else ""

def assert_same(x, y, line, dtype):
    assert(x.dtype == dtype)
    try: torch.testing.assert_close(x, y, check_stride = True)
    except Exception as error:
        raise RuntimeError(
            f"Failed allclose at line [{line}]: {NAME(x)}, {NAME(y)}\n{str(error)}"
        )

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# A) Convert `nf4` to Triton. (Original Challenge)

<a name="NF4"></a>



1. Goal: Convert a `nf4` quantized tensor into `fp16` or `bf16` into a *single* Triton kernel The double dequant of the `absmax` and weight forming must be done in 1 Triton kernel. Must work on Tesla T4.
2. Must be faster than Unsloth's `fast_dequantize` by 1.15x or more, and not use large intermediate memory buffers.
3. Must not use `torch.compile`, but can use `trace.enabled` to help on writing Triton kernels.
4. Good material: [Unsloth `fast_dequantize` function](https://github.com/unslothai/unsloth/blob/main/unsloth/kernels/utils.py#L128), also [bitsandbytes `dequantize_blockwise`](https://github.com/bitsandbytes-foundation/bitsandbytes/blob/86b6c37a8ad448230cedb60753f63150b603a112/bitsandbytes/functional.py#L958)
5. Use `test_dequantize_function` to test your implementation.
6. No CUDA allowed. Custom CUDA inside of the Triton is allowed.
7. Watch Tim's videos on Youtube: [8-bit Optimizers](https://www.youtube.com/watch?v=2ETNONas068)

In [22]:
from bitsandbytes.nn import Linear4bit
from transformers.activations import ACT2FN
from unsloth.kernels.utils import fast_dequantize
from peft.utils.integrations import dequantize_module_weight as peft_dequantize
def unsloth_dequantize(weight):
    return fast_dequantize(weight.weight, weight.weight.quant_state)

def bnb_Linear4bit(hd, m, dtype = torch.float16):
    return Linear4bit(
        hd, m, bias = None,
        compute_dtype       = dtype,
        compress_statistics = True,
        quant_type          = "nf4",
    )

# [NEW] as at 18th Feb 2025
def assert_correct_bnb(weight, dtype):
    assert(weight.weight.dtype == torch.uint8)
    assert(weight.weight.quant_state.dtype == dtype)
    assert(weight.weight.quant_state.absmax.dtype == torch.uint8)
    assert(weight.weight.quant_state.code.dtype == torch.float32)
    assert(weight.weight.quant_state.offset.dtype == torch.float32)
    assert(weight.weight.quant_state.blocksize == 64)
    assert(weight.weight.quant_state.state2.absmax.dtype == torch.float32)
    assert(weight.weight.quant_state.state2.code.dtype == torch.float32)
    assert(weight.weight.quant_state.state2.blocksize == 256)

class MLP(nn.Module):
    def __init__(self, hd = 4096, m = 14336, dtype = torch.float16):
        super().__init__()
        self.gate_proj = bnb_Linear4bit(hd, m, dtype = dtype).to("cuda")
        self.up_proj   = bnb_Linear4bit(hd, m, dtype = dtype).to("cuda")
        self.down_proj = bnb_Linear4bit(m, hd, dtype = dtype).to("cuda")
        # [NEW] as at 18th Feb 2025
        self.gate_proj.weight.quant_state.dtype = dtype
        self.up_proj  .weight.quant_state.dtype = dtype
        self.down_proj.weight.quant_state.dtype = dtype
        self.act_fn = ACT2FN["silu"]
    def forward(self, x):
        return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))

def mlp_forward(X, mlp, fx):
    up   = X @ fx(mlp.  up_proj).t()
    gate = X @ fx(mlp.gate_proj).t()
    h = mlp.act_fn(gate) * up
    down = h @ fx(mlp.down_proj).t()
    return down

def mlp_dequantize(X, mlp, fx):
    a = fx(mlp.  up_proj).t(); torch.cuda.synchronize()
    b = fx(mlp.gate_proj).t(); torch.cuda.synchronize()
    c = fx(mlp.down_proj).t(); torch.cuda.synchronize()
    return a, b, c

def test_dequantize(dequantize_fx):
    elapsed = 0
    options = [
        (2, 3333, 2048,  8192, 3407, torch.float16),
        (5,  777, 1024,  4096, 3409, torch.bfloat16),
        (3, 2048, 4096, 14336, 3408, torch.bfloat16),
    ]
    for (bsz, qlen, hd, m, seed, dt) in options:
        set_seed(seed)
        torch.set_default_dtype(torch.float32)
        mlp = MLP(hd = hd, m = m, dtype = dt)
        X = torch.randn((bsz, qlen, hd), device = "cuda", dtype = dt)
        torch.cuda.synchronize()

        # Warmup
        for _ in range(2):
            assert_same( mlp_forward(X, mlp, dequantize_fx), mlp(X), _F(_C()), dt)
            # [NEW] as at 18th Feb 2025
            assert_correct_bnb(mlp.  up_proj, dt)
            assert_correct_bnb(mlp.gate_proj, dt)
            assert_correct_bnb(mlp.down_proj, dt)
            a, b, c = mlp_dequantize(X, mlp, dequantize_fx)
            A, B, C = mlp_dequantize(X, mlp, unsloth_dequantize)
            assert_same(a, A, _F(_C()), dt)
            assert_same(b, B, _F(_C()), dt)
            assert_same(c, C, _F(_C()), dt)

        # Benchmarking
        torch.cuda.synchronize()
        start = time.time()
        for _ in range(1000): mlp_dequantize(X, mlp, dequantize_fx)
        elapsed += time.time() - start
    return elapsed

For example, we can test our implementation via:

In [18]:
from unsloth.kernels.utils import fast_dequantize
def unsloth_dequantize(weight):
    return fast_dequantize(weight.weight, weight.weight.quant_state)
test_dequantize(unsloth_dequantize)

3.02012038230896

NOTE: The above time is on a L4 GPU On a T4 it comes out at 3.9 s

The elapsed time for our implementation over 1000 trials is 5.38 seconds or so.

PEFT also has one, which should be mostly identical to Unsloth's version, albeit slightly slower.

In [19]:
from peft.utils.integrations import dequantize_module_weight as peft_dequantize
test_dequantize(peft_dequantize)

3.842420816421509

The above time is on a L4

# Unsloth Harness

Write your Triton kernel below, and test it:

In [ ]:
from triton import jit
import triton
import triton.language as tl

@triton.jit
def _your_dequantize_nf4_kernel():
    ### TRITON CODE GOES HERE
    return

def _your_dequantize_nf4(weight, quant_state):
    ### SETUP TRITON LAUNCH HERE
    return None

def your_dequantize_nf4(weight):
    return _your_dequantize_nf4(weight.weight.data, weight.weight.quant_state)

In [ ]:
### TEST IT BELOW:
# test_dequantize(your_dequantize_nf4)

### CALCULATE SPEEDUP (hopefully 1.15x faster or more)
# test_dequantize(unsloth_dequantize) / test_dequantize(your_dequantize_nf4)

## Marking Criteria for A) Max points = 14
```python
if attemped_A:
    A_score = 0
    if single_triton_kernel: A_score += 3
    speedup = old_time / new_time
    if speedup <= 1.00: A_score -= 3
    if speedup >= 1.05: A_score += 1
    if speedup >= 1.10: A_score += 2
    if speedup >= 1.15: A_score += 2
    if kernel_works_in_torch_compile: A_score += 1
    else: A_score -= 1
    if custom_asm_works: A_score += 3
    if uses_cache_eviction: A_score += 1
    if tested_in_f16_and_bf16: A_score += 1
    else: A_score -= 1
    final_score += A_score
else:
    final_score += 0
```

# NF4 → fp16 Dequant in Mojo (Unsloth Puzzle A, Mojo Edition – T4 vs L4)

This is a Mojo GPU implementation of Unsloth’s **“Convert `nf4` to Triton”** puzzle (Task A), but written as straight Mojo kernels instead of Triton/CUDA.

> **Goal:** Dequantize bitsandbytes-style `nf4` weights (with double-quant stats) into `fp16` using a **single custom GPU kernel**, no `torch.compile`, and benchmark under the same workload as Unsloth’s `test_dequantize`.

### What the kernels do

All variants implement full **bitsandbytes / Unsloth NF4** semantics:

* `blocksize = 64` (64 weights per NF4 block, 32 bytes of codes)
* `state2.blocksize = 256` (256 blocks per outer scale)
* Standard NF4 16-entry codebook
* Double dequantization:

  * `absmax_q` → `state2.code` → `state2.absmax` → `offset`

Two “final” Mojo kernels:

1. **Warp-per-NF4-block kernel (shared scale + packed store)**

   * 256 threads per block (8 warps).
   * **1 warp = 1 NF4 block** (64 weights = 32 bytes).
   * Lane 0 per warp dequantizes the block’s fp32 scale once and writes it to shared memory; all 32 lanes reuse it.
   * Each lane handles **1 NF4 byte → 2 `fp16` values**, packs them into a single `u32`, and does **one 32-bit store**.

2. **2D tiled kernel (1024 threads + packed store)**

   * 2D tiling over `(rows, cols)` with `256 × 4 = 1024` threads per block.
   * **1 thread = 1 NF4 byte → 2 `fp16` values → 1 `u32` store**.
   * Recomputes the scale per element (no shared scale), but has very regular memory access and high occupancy.

Both are **plain Mojo** using `DeviceContext` – no C++, no raw CUDA, no `torch.compile`.

---

### Benchmark harness (exact same workload as Unsloth)

I reuse Unsloth’s `test_dequantize` structure, but replace `fast_dequantize` with my Mojo kernel wrapper:

* Three `(hd, m)` configs:

  * `(2048, 8192)`
  * `(1024, 4096)`
  * `(4096, 14336)`
* For each config, dequantize three matrices:

  * `up_proj   : (m, hd)`
  * `gate_proj : (m, hd)`
  * `down_proj : (hd, m)`
* Each matrix: **1000 dequant calls**, with `torch.cuda.synchronize()` in the original harness.
* Total: **9000 dequants** per run.
* For Mojo, I mirror this: same shapes, same 3×3×1000 structure, and time only the kernel launches using `ctx.execution_time` with fixed device buffers.

Correctness is checked twice:

* Mojo CPU reference vs Mojo GPU (bit-accurate in fp16).
* Earlier in the process, Mojo GPU vs Unsloth’s `fast_dequantize` outputs.

---

### T4 vs L4: performance comparison

Below are **end-to-end `test_dequantize` workloads** (3 configs × 3 matrices × 1000 calls), measured on Colab T4 and L4:

#### Baselines

* **Unsloth Triton reference** (original notebook, T4):
  ≈ **5.32 s**
* **Unsloth `fast_dequantize` (CUDA C)**

  * T4: ≈ **3.90 s** (from Unsloth’s updated note)
  * L4: measured ≈ **3.02 s**
* **PEFT dequant (CUDA C)** on L4: ≈ **3.84 s**

#### Mojo kernels – T4 (Tesla T4)

* **Warp-per-NF4-block (shared scale + packed store)**
  → **≈ 4.50 s** total
  → **speedup vs Triton ref ~1.18×**

* **2D tiled + packed 32-bit store (1024 threads/block)**
  → **≈ 4.27 s** total
  → **speedup vs Triton ref ~1.24×**
  → Within ~10% of Unsloth’s internal CUDA fast path (3.9 s) on this GPU.

On T4 the **2D packed kernel wins**: large CTAs and very simple per-thread control flow beat the warp-per-block design, even though that one does less math per element.

#### Mojo kernels – L4 (NVIDIA L4)

Same harness, just on L4:

* **Unsloth `fast_dequantize` (CUDA C)**
  → `test_dequantize(unsloth_dequantize) ≈ 3.02 s`

* **Warp-per-NF4-block (shared scale + packed store)**
  → **≈ 2.47 s** total
  → **speedup vs Triton ref ~2.15×**
  → **speedup vs CUDA fast_dequantize ~1.22×**

* **2D tiled + packed 32-bit store (1024 threads/block)**
  → **≈ 2.60 s** total
  → **speedup vs Triton ref ~2.05×**
  → **speedup vs CUDA fast_dequantize ~1.16×**

On **L4**, the ranking **flips**:

* The **warp-per-NF4-block kernel** becomes the fastest:

  * 1 warp = 1 NF4 block,
  * shared scale per block,
  * one packed 32-bit store per byte.
* The 2D packed kernel is still very strong, but now slightly behind the warp version.

In terms of effective bandwidth on the largest config (4096×14336 ≈ 58.7M weights), the warp kernel is in the **200–250 GB/s** ballpark on L4, which is right where a memory-bound kernel should be on this card.

---

### TL;DR

* On **T4**, the 2D 1024-thread Mojo kernel with packed `u32` stores beats the Triton reference by ~**1.2–1.3×** and gets close to Unsloth’s CUDA fast path.
* On **L4**, both Mojo kernels are **2× faster** than the original Triton harness, and the **warp-per-NF4-block** Mojo kernel is actually **~1.2× faster** than Unsloth’s current CUDA `fast_dequantize`.
* All of this is done in **pure Mojo**, no C++/CUDA and no `torch.compile` just hand-written kernels and a simple timing harness that mirrors the original `test_dequantize` structure.

For me, this was mainly a “can I do this in Mojo, and how close can I get” experiment. The fun twist is that on L4 the answer turned out to be: *closer than expected, and in some cases actually ahead of the C/CUDA implementation it was inspired by.*

## Also of Note

The bitsandbytes’ CUDA fast path is implemented as **two kernels** (one to dequantize the stats into fp32 scales, and a second to dequantize the NF4 weights), whereas the Mojo kernels here fuse both stages into a **single GPU kernel** with no large intermediate buffers, and still comes out faster on an L4 for the same `test_dequantize` workload.


## Background and Reason

I'm a software engineer with a core background in web development and some Python, and I’ve always had a strong drive to understand how things work at a deeper level. When I first learned about Mojo and the Modular team’s mission to make GPU programming more approachable, I started following their progress closely.

Lately I’ve been experimenting with max, custom ops, and other lower-level pieces of the stack. When I came across this notebook, I decided to treat it as a personal challenge: could someone without prior GPU-kernel experience pick up Mojo and become productive quickly, and could that same person build something competitive with highly optimized, hand-tuned kernels?

After roughly seven hours of work, I’m happy with the result. The kernel I built is  competitive on T4 and 1.3x faster on L4. The process reinforced my belief that Mojo successfully lowers the barrier to entry by pairing a Python-like feel with low-level control when needed.


David Robertson  
info@drobetson.pro  
github.com/daver987

# Install mojo

In [1]:
!pip install mojo --index-url https://dl.modular.com/public/nightly/python/simple/

Looking in indexes: https://dl.modular.com/public/nightly/python/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 100.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 MB 9.9 MB/s eta 0:00:00


### Check that mojo is available and import notebook

In [2]:
import mojo.notebook

### Check that we are on the GPU

In [3]:
%%mojo

from gpu.host import DeviceContext

fn kernel():
    print("Hello from the GPU")

def main():
    with DeviceContext() as ctx:
        ctx.enqueue_function_checked[kernel, kernel](grid_dim=1, block_dim=1)
        ctx.synchronize()

Hello from the GPU



# Kernel - 2D Tiling 32 bit Store **L4** GPU

In [21]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation, bitcast
from math import fma, ceildiv

#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16
comptime u32_dtype = DType.uint32
comptime u16_dtype = DType.uint16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]
comptime U32 = Scalar[u32_dtype]
comptime U16 = Scalar[u16_dtype]

# Bitsandbytes NF4 codebook
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block
comptime BLOCKSIZE_W_SHIFT = 6  # log2(64) for bit shift
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale
comptime STATE2_SHIFT = 8  # log2(256) for bit shift

# 2D tiling: each thread handles 1 byte = 2 weights
# Output as packed 32-bit (2 x fp16)
comptime TILE_BYTES_X = 256  # bytes per block in X (= 512 weights)
comptime TILE_ROWS = 4  # rows per block
comptime THREADS_X = 256
comptime THREADS_Y = 4
# Total: 1024 threads per block, each outputs 32 bits (2 fp16)

# ----------------------------------------------------
# GPU kernel: PACKED 32-bit STORE
#   - Each thread handles 1 packed byte = 2 weights
#   - Outputs a single 32-bit word (2 x fp16 packed)
#   - 256×4 = 1024 threads per CUDA block
#   - Shared memory for NF4 codebook
# ---------------------------------------------------


fn nf4_dequant_kernel_packed(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[U32, MutAnyOrigin],  # Output as packed 32-bit words
    n_rows: Int,
    n_bytes_per_row: Int,  # n_cols // 2
    blocks_per_row: Int,
    groups_per_row: Int,
):
    # Shared memory for NF4 codebook (16 values)
    var shared_nf4 = stack_allocation[
        16, F32, address_space = AddressSpace.SHARED
    ]()

    # First 16 threads of first row load the NF4 codebook
    var tx = Int(thread_idx.x)
    var ty = Int(thread_idx.y)
    if ty == 0 and tx < 16:
        shared_nf4[tx] = NF4_TABLE[tx]

    barrier()

    # 2D position: (row, byte_idx within row)
    var row = Int(block_idx.x) * TILE_ROWS + ty
    var byte_idx = Int(block_idx.y) * TILE_BYTES_X + tx

    if row >= n_rows or byte_idx >= n_bytes_per_row:
        return

    # Read the packed byte
    var packed_byte: U8 = packed_ptr[row * n_bytes_per_row + byte_idx]
    var packed_int: Int = Int(packed_byte)

    # Extract low and high nibbles
    var lo_idx: Int = packed_int & 0x0F
    var hi_idx: Int = (packed_int >> 4) & 0x0F

    # NF4 lookup from shared memory
    var v0_norm: F32 = shared_nf4[lo_idx]
    var v1_norm: F32 = shared_nf4[hi_idx]

    # Column index for weight 0 (col0 = byte_idx * 2)
    var col0 = byte_idx << 1  # byte_idx * 2

    # Compute block/group indices using bit shifts
    var block_id = col0 >> BLOCKSIZE_W_SHIFT  # col0 // 64
    var absmax_idx = row * blocks_per_row + block_id

    # First-level dequant: absmax_q -> code2 lookup
    var q_scale: U8 = absmax_q_ptr[absmax_idx]
    var code_val: F32 = code2_ptr[Int(q_scale)]

    # Second-level dequant: group scale
    var group_id = block_id >> STATE2_SHIFT  # block_id // 256
    var group_idx = row * groups_per_row + group_id
    var scale_base: F32 = scale2_ptr[group_idx]

    # Final scale = code2[absmax_q] * scale2 + offset
    var scale: F32 = fma(code_val, scale_base, offset)

    # Dequantized values
    var v0: F32 = v0_norm * scale
    var v1: F32 = v1_norm * scale

    # Convert to fp16
    var h0: F16 = F16(v0)
    var h1: F16 = F16(v1)

    # Bitcast to u16
    var b0: U16 = bitcast[u16_dtype](h0)
    var b1: U16 = bitcast[u16_dtype](h1)

    # Pack into 32-bit word: [h1:h0] (h0 in low 16 bits, h1 in high 16 bits)
    var packed_out: U32 = (U32(b1) << 16) | U32(b0)

    # Output word index (one 32-bit word per byte = 2 weights)
    # word_stride = n_bytes_per_row (since each byte -> one 32-bit word)
    var word_idx = row * n_bytes_per_row + byte_idx

    out_ptr[word_idx] = packed_out


# -----------------------------
# CPU reference (same logic, stores as fp16 pairs)
# -----------------------------


fn nf4_dequant_cpu_packed(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[
        F16, MutAnyOrigin
    ],  # CPU uses F16 for easier verification
    n_rows: Int,
    n_cols: Int,
):
    var n_bytes_per_row = n_cols >> 1
    var blocks_per_row = (n_cols + BLOCKSIZE_W - 1) >> BLOCKSIZE_W_SHIFT
    var groups_per_row = (blocks_per_row + STATE2_BLOCKSIZE - 1) >> STATE2_SHIFT

    for row in range(n_rows):
        var row_packed_base = row * n_bytes_per_row
        var row_out_base = row * n_cols
        var row_absmax_base = row * blocks_per_row
        var row_group_base = row * groups_per_row

        for byte_idx in range(n_bytes_per_row):
            var packed_byte: U8 = packed[row_packed_base + byte_idx]
            var packed_int: Int = Int(packed_byte)

            var lo_idx: Int = packed_int & 0x0F
            var hi_idx: Int = (packed_int >> 4) & 0x0F

            var v0_norm: F32 = NF4_TABLE[lo_idx]
            var v1_norm: F32 = NF4_TABLE[hi_idx]

            var col0 = byte_idx << 1
            var block_id = col0 >> BLOCKSIZE_W_SHIFT
            var absmax_idx = row_absmax_base + block_id

            var q_scale: U8 = absmax_q[absmax_idx]
            var code_val: F32 = code2[Int(q_scale)]

            var group_id = block_id >> STATE2_SHIFT
            var group_idx = row_group_base + group_id
            var scale_base: F32 = scale2[group_idx]

            var scale: F32 = fma(code_val, scale_base, offset)

            var v0: F32 = v0_norm * scale
            var v1: F32 = v1_norm * scale

            output[row_out_base + col0] = F16(v0)
            output[row_out_base + col0 + 1] = F16(v1)


# -----------------------------
# Correctness check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    # Use a small 2D test case: 4 rows × 128 cols
    var n_rows: Int = 4
    var n_cols: Int = 128
    var n_weights: Int = n_rows * n_cols
    var n_bytes: Int = n_weights // 2
    var n_bytes_per_row: Int = n_cols // 2

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    nf4_dequant_cpu_packed(
        packed_host.unsafe_ptr(),
        absmax_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        scale2_host.unsafe_ptr(),
        F32(offset),
        out_cpu_host.unsafe_ptr(),
        n_rows,
        n_cols,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    # Output buffer: n_bytes 32-bit words (each = 2 fp16)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # 2D grid: grid_x = rows/TILE_ROWS, grid_y = bytes_per_row/TILE_BYTES_X
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    ctx.enqueue_function_checked[
        nf4_dequant_kernel_packed, nf4_dequant_kernel_packed
    ](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_rows,
        n_bytes_per_row,
        blocks_per_row,
        groups_per_row,
        grid_dim=(grid_x, grid_y),
        block_dim=(THREADS_X, THREADS_Y),
    )
    ctx.synchronize()

    # Compare: unpack 32-bit GPU output to fp16 pairs
    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_bytes):
            var packed_word: U32 = out_gpu_host[i]
            var b0: U16 = U16(packed_word & 0xFFFF)
            var b1: U16 = U16((packed_word >> 16) & 0xFFFF)
            var gpu_v0: F16 = bitcast[f16_dtype](b0)
            var gpu_v1: F16 = bitcast[f16_dtype](b1)

            var idx0 = i * 2
            var idx1 = idx0 + 1

            var cpu_v0: F32 = F32(out_cpu_host[idx0])
            var cpu_v1: F32 = F32(out_cpu_host[idx1])
            var gpu_f0: F32 = F32(gpu_v0)
            var gpu_f1: F32 = F32(gpu_v1)

            var diff0: F32 = cpu_v0 - gpu_f0
            var diff1: F32 = cpu_v1 - gpu_f1

            if diff0 < 0.0:
                diff0 = -diff0
            if diff1 < 0.0:
                diff1 = -diff1

            if diff0 > max_abs_diff:
                max_abs_diff = diff0
            if diff1 > max_abs_diff:
                max_abs_diff = diff1

            if i < 8:
                print(idx0, "\t", cpu_v0, "\t", gpu_f0, "\t", cpu_v0 - gpu_f0)
                print(idx1, "\t", cpu_v1, "\t", gpu_f1, "\t", cpu_v1 - gpu_f1)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# -----------------------------


fn bench_layer_packed(
    ctx: DeviceContext,
    n_rows: Int,
    n_cols: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Benchmark packed 32-bit store kernel.
    Returns: seconds (total time for num_iters iterations)
    """
    var n_weights = n_rows * n_cols
    var n_bytes = n_weights // 2
    var n_bytes_per_row = n_cols // 2

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # 2D grid
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel_packed, nf4_dequant_kernel_packed
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_rows,
            n_bytes_per_row,
            blocks_per_row,
            groups_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Mojo Packed 32-bit Store) L4 GPU")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print(
            "Using packed 32-bit store kernel: ",
            THREADS_X,
            "x",
            THREADS_Y,
            " = 1024 threads/block",
        )
        print("Each thread: 1 byte -> 2 fp16 -> 1 x 32-bit store")
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        print("\nConfig 1: rows=", hd1, ", cols=", m1, ", n_weights=", hd1 * m1)

        var t1_up = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        print("\nConfig 2: rows=", hd2, ", cols=", m2, ", n_weights=", hd2 * m2)

        var t2_up = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        print("\nConfig 3: rows=", hd3, ", cols=", m3, ", n_weights=", hd3 * m3)

        var t3_up = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n✅ SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n❌ Need more optimization to reach 1.15x speedup")


NF4 Dequantization Benchmark (Mojo Packed 32-bit Store) L4 GPU

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.010002136 	 -0.010002136 	 0.0
1 	 -0.0069618225 	 -0.0069618225 	 0.0
2 	 -0.0069618225 	 -0.0069618225 	 0.0
3 	 -0.0052490234 	 -0.0052490234 	 0.0
4 	 -0.0052490234 	 -0.0052490234 	 0.0
5 	 -0.0039482117 	 -0.0039482117 	 0.0
6 	 -0.0039482117 	 -0.0039482117 	 0.0
7 	 -0.0028438568 	 -0.0028438568 	 0.0
8 	 -0.0028438568 	 -0.0028438568 	 0.0
9 	 -0.0018472672 	 -0.0018472672 	 0.0
10 	 -0.0018472672 	 -0.0018472672 	 0.0
11 	 -0.00091028214 	 -0.00091028214 	 0.0
12 	 -0.00091028214 	 -0.00091028214 	 0.0
13 	 0.0 	 0.0 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.0007958412 	 0.0007958412 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.
Using packed 32-bit store kernel:  256 x 4  = 1024 threads/block
Each thread: 1 byte -> 2 fp16 -> 1 x 32-bit store

Device: cuda

Config 1: rows= 2048 , cols= 8192 , n

# Warp per NF4 block **L4** GPU

In [20]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation, bitcast
from math import fma, ceildiv

# -----------------------------
# Types and constants
# -----------------------------

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16
comptime u32_dtype = DType.uint32
comptime u16_dtype = DType.uint16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]
comptime U32 = Scalar[u32_dtype]
comptime U16 = Scalar[u16_dtype]

# Bitsandbytes NF4 codebook
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block = 32 bytes
comptime BYTES_PER_NF4_BLOCK = 32  # 64 weights / 2 = 32 bytes
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale
comptime STATE2_SHIFT = 8  # log2(256) for bit shift

# Warp-based tiling:
# - 1 warp (32 threads) = 1 NF4 block (32 bytes = 64 weights)
# - Each thread handles 1 byte = 2 weights
# - Lane 0 computes scale, broadcasts via shared memory
comptime WARP_SIZE = 32
comptime WARPS_PER_BLOCK = 8  # 8 warps per CUDA block = 256 threads
comptime THREADS_PER_BLOCK = WARP_SIZE * WARPS_PER_BLOCK  # 256

# Each CUDA block handles 8 NF4 blocks (8 warps × 32 bytes = 256 bytes = 512 weights)

# ------------------------------------------------------------------------
# GPU kernel: WARP-PER-NF4-BLOCK with shared scale
#   - 1 warp = 1 NF4 block (32 bytes = 64 weights)
#   - Lane 0 computes scale once, stores to shared_scale[warp_id]
#   - All 32 lanes read scale from shared, unpack, multiply, pack, store
#   - Eliminates 31/32 redundant scale loads/computes per NF4 block
# ------------------------------------------------------------------------


fn nf4_dequant_kernel_warp(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[U32, MutAnyOrigin],
    n_nf4_blocks: Int,  # total NF4 blocks = n_weights / 64
    n_bytes: Int,  # total bytes = n_weights / 2
    groups_per_row: Int,  # for 1D layout: ceildiv(n_absmax, STATE2_BLOCKSIZE)
):
    # Shared memory for NF4 codebook (16 values)
    var shared_nf4 = stack_allocation[
        16, F32, address_space = AddressSpace.SHARED
    ]()
    # Shared memory for per-warp scales (one per warp in this CUDA block)
    var shared_scale = stack_allocation[
        WARPS_PER_BLOCK, F32, address_space = AddressSpace.SHARED
    ]()

    var tid = Int(thread_idx.x)
    var warp_id = tid >> 5  # tid // 32
    var lane_id = tid & 31  # tid % 32

    # First 16 threads load the NF4 codebook
    if tid < 16:
        shared_nf4[tid] = NF4_TABLE[tid]

    barrier()

    # Global NF4 block index for this warp
    var nf4_block_idx = Int(block_idx.x) * WARPS_PER_BLOCK + warp_id

    if nf4_block_idx >= n_nf4_blocks:
        return

    # Lane 0 computes the scale for this NF4 block
    if lane_id == 0:
        # absmax_q for this NF4 block
        var q_scale: U8 = absmax_q_ptr[nf4_block_idx]
        var code_val: F32 = code2_ptr[Int(q_scale)]

        # Group index for second-level scale
        var group_id = nf4_block_idx >> STATE2_SHIFT  # nf4_block_idx // 256
        var scale_base: F32 = scale2_ptr[group_id]

        # Final scale = code2[absmax_q] * scale2 + offset
        var scale: F32 = fma(code_val, scale_base, offset)
        shared_scale[warp_id] = scale

    barrier()

    # All lanes read the scale from shared memory
    var scale: F32 = shared_scale[warp_id]

    # Global byte index for this thread
    var byte_idx = nf4_block_idx * BYTES_PER_NF4_BLOCK + lane_id

    if byte_idx >= n_bytes:
        return

    # Read the packed byte
    var packed_byte: U8 = packed_ptr[byte_idx]
    var packed_int: Int = Int(packed_byte)

    # Extract low and high nibbles
    var lo_idx: Int = packed_int & 0x0F
    var hi_idx: Int = (packed_int >> 4) & 0x0F

    # NF4 lookup from shared memory
    var v0_norm: F32 = shared_nf4[lo_idx]
    var v1_norm: F32 = shared_nf4[hi_idx]

    # Dequantized values (just multiply by scale - no per-thread loads!)
    var v0: F32 = v0_norm * scale
    var v1: F32 = v1_norm * scale

    # Convert to fp16
    var h0: F16 = F16(v0)
    var h1: F16 = F16(v1)

    # Bitcast to u16
    var b0: U16 = bitcast[u16_dtype](h0)
    var b1: U16 = bitcast[u16_dtype](h1)

    # Pack into 32-bit word: [h1:h0]
    var packed_out: U32 = (U32(b1) << 16) | U32(b0)

    # Store (one 32-bit word per byte)
    out_ptr[byte_idx] = packed_out


# -----------------------------
# CPU reference (flat 1D layout)
# -----------------------------


fn nf4_dequant_cpu(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[F16, MutAnyOrigin],
    n_bytes: Int,
):
    for byte_idx in range(n_bytes):
        var packed_byte: U8 = packed[byte_idx]
        var packed_int: Int = Int(packed_byte)

        var lo_idx: Int = packed_int & 0x0F
        var hi_idx: Int = (packed_int >> 4) & 0x0F

        var v0_norm: F32 = NF4_TABLE[lo_idx]
        var v1_norm: F32 = NF4_TABLE[hi_idx]

        # NF4 block index for this byte
        var nf4_block_idx = byte_idx >> 5  # byte_idx // 32

        var q_scale: U8 = absmax_q[nf4_block_idx]
        var code_val: F32 = code2[Int(q_scale)]

        var group_id = nf4_block_idx >> STATE2_SHIFT
        var scale_base: F32 = scale2[group_id]

        var scale: F32 = fma(code_val, scale_base, offset)

        var v0: F32 = v0_norm * scale
        var v1: F32 = v1_norm * scale

        var out_idx = byte_idx << 1  # byte_idx * 2
        output[out_idx] = F16(v0)
        output[out_idx + 1] = F16(v1)


# -----------------------------
# Correctness check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    # Small test: 512 weights = 8 NF4 blocks = 256 bytes
    var n_weights: Int = 512
    var n_bytes: Int = n_weights // 2
    var n_nf4_blocks: Int = n_weights // BLOCKSIZE_W
    var n_scale2: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var groups_per_row: Int = n_scale2  # 1D layout

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_nf4_blocks):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    nf4_dequant_cpu(
        packed_host.unsafe_ptr(),
        absmax_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        scale2_host.unsafe_ptr(),
        F32(offset),
        out_cpu_host.unsafe_ptr(),
        n_bytes,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # Grid: each CUDA block handles WARPS_PER_BLOCK NF4 blocks
    var grid_size: Int = ceildiv(n_nf4_blocks, WARPS_PER_BLOCK)

    ctx.enqueue_function_checked[
        nf4_dequant_kernel_warp, nf4_dequant_kernel_warp
    ](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_nf4_blocks,
        n_bytes,
        groups_per_row,
        grid_dim=grid_size,
        block_dim=THREADS_PER_BLOCK,
    )
    ctx.synchronize()

    # Compare
    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_bytes):
            var packed_word: U32 = out_gpu_host[i]
            var b0: U16 = U16(packed_word & 0xFFFF)
            var b1: U16 = U16((packed_word >> 16) & 0xFFFF)
            var gpu_v0: F16 = bitcast[f16_dtype](b0)
            var gpu_v1: F16 = bitcast[f16_dtype](b1)

            var idx0 = i * 2
            var idx1 = idx0 + 1

            var cpu_v0: F32 = F32(out_cpu_host[idx0])
            var cpu_v1: F32 = F32(out_cpu_host[idx1])
            var gpu_f0: F32 = F32(gpu_v0)
            var gpu_f1: F32 = F32(gpu_v1)

            var diff0: F32 = cpu_v0 - gpu_f0
            var diff1: F32 = cpu_v1 - gpu_f1

            if diff0 < 0.0:
                diff0 = -diff0
            if diff1 < 0.0:
                diff1 = -diff1

            if diff0 > max_abs_diff:
                max_abs_diff = diff0
            if diff1 > max_abs_diff:
                max_abs_diff = diff1

            if i < 8:
                print(idx0, "\t", cpu_v0, "\t", gpu_f0, "\t", cpu_v0 - gpu_f0)
                print(idx1, "\t", cpu_v1, "\t", gpu_f1, "\t", cpu_v1 - gpu_f1)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# -----------------------------


fn bench_layer_warp(
    ctx: DeviceContext,
    n_weights: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Benchmark warp-per-NF4-block kernel.
    Returns: seconds (total time for num_iters iterations)
    """
    var n_bytes = n_weights // 2
    var n_nf4_blocks = n_weights // BLOCKSIZE_W
    var n_scale2 = ceildiv(n_nf4_blocks, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var groups_per_row = n_scale2

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_nf4_blocks):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # Grid size
    var grid_size: Int = ceildiv(n_nf4_blocks, WARPS_PER_BLOCK)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel_warp, nf4_dequant_kernel_warp
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_nf4_blocks,
            n_bytes,
            groups_per_row,
            grid_dim=grid_size,
            block_dim=THREADS_PER_BLOCK,
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Warp-per-NF4-Block) L4")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print("Kernel: 1 warp = 1 NF4 block, lane 0 computes shared scale")
        print(
            "Threads per block:",
            THREADS_PER_BLOCK,
            "(",
            WARPS_PER_BLOCK,
            "warps)",
        )
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        var n_w1: Int = hd1 * m1
        print("\nConfig 1: hd=", hd1, ", m=", m1, ", n_weights=", n_w1)

        var t1_up = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        var n_w2: Int = hd2 * m2
        print("\nConfig 2: hd=", hd2, ", m=", m2, ", n_weights=", n_w2)

        var t2_up = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        var n_w3: Int = hd3 * m3
        print("\nConfig 3: hd=", hd3, ", m=", m3, ", n_weights=", n_w3)

        var t3_up = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n Need more optimization to reach 1.15x speedup")


NF4 Dequantization Benchmark (Warp-per-NF4-Block) L4

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.010002136 	 -0.010002136 	 0.0
1 	 -0.0069618225 	 -0.0069618225 	 0.0
2 	 -0.0069618225 	 -0.0069618225 	 0.0
3 	 -0.0052490234 	 -0.0052490234 	 0.0
4 	 -0.0052490234 	 -0.0052490234 	 0.0
5 	 -0.0039482117 	 -0.0039482117 	 0.0
6 	 -0.0039482117 	 -0.0039482117 	 0.0
7 	 -0.0028438568 	 -0.0028438568 	 0.0
8 	 -0.0028438568 	 -0.0028438568 	 0.0
9 	 -0.0018472672 	 -0.0018472672 	 0.0
10 	 -0.0018472672 	 -0.0018472672 	 0.0
11 	 -0.00091028214 	 -0.00091028214 	 0.0
12 	 -0.00091028214 	 -0.00091028214 	 0.0
13 	 0.0 	 0.0 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.0007958412 	 0.0007958412 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.
Kernel: 1 warp = 1 NF4 block, lane 0 computes shared scale
Threads per block: 256 ( 8 warps)

Device: cuda

Config 1: hd= 2048 , m= 8192 , n_weights= 16777216
  up_proj   :  0.1

# Kernel - 2D Tiling 32 bit Store **T4** GPU

In [4]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation, bitcast
from math import fma, ceildiv

#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16
comptime u32_dtype = DType.uint32
comptime u16_dtype = DType.uint16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]
comptime U32 = Scalar[u32_dtype]
comptime U16 = Scalar[u16_dtype]

# Bitsandbytes NF4 codebook
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block
comptime BLOCKSIZE_W_SHIFT = 6  # log2(64) for bit shift
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale
comptime STATE2_SHIFT = 8  # log2(256) for bit shift

# 2D tiling: each thread handles 1 byte = 2 weights
# Output as packed 32-bit (2 x fp16)
comptime TILE_BYTES_X = 256  # bytes per block in X (= 512 weights)
comptime TILE_ROWS = 4  # rows per block
comptime THREADS_X = 256
comptime THREADS_Y = 4
# Total: 1024 threads per block, each outputs 32 bits (2 fp16)

# ----------------------------------------------------
# GPU kernel: PACKED 32-bit STORE
#   - Each thread handles 1 packed byte = 2 weights
#   - Outputs a single 32-bit word (2 x fp16 packed)
#   - 256×4 = 1024 threads per CUDA block
#   - Shared memory for NF4 codebook
# ---------------------------------------------------


fn nf4_dequant_kernel_packed(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[U32, MutAnyOrigin],  # Output as packed 32-bit words
    n_rows: Int,
    n_bytes_per_row: Int,  # n_cols // 2
    blocks_per_row: Int,
    groups_per_row: Int,
):
    # Shared memory for NF4 codebook (16 values)
    var shared_nf4 = stack_allocation[
        16, F32, address_space = AddressSpace.SHARED
    ]()

    # First 16 threads of first row load the NF4 codebook
    var tx = Int(thread_idx.x)
    var ty = Int(thread_idx.y)
    if ty == 0 and tx < 16:
        shared_nf4[tx] = NF4_TABLE[tx]

    barrier()

    # 2D position: (row, byte_idx within row)
    var row = Int(block_idx.x) * TILE_ROWS + ty
    var byte_idx = Int(block_idx.y) * TILE_BYTES_X + tx

    if row >= n_rows or byte_idx >= n_bytes_per_row:
        return

    # Read the packed byte
    var packed_byte: U8 = packed_ptr[row * n_bytes_per_row + byte_idx]
    var packed_int: Int = Int(packed_byte)

    # Extract low and high nibbles
    var lo_idx: Int = packed_int & 0x0F
    var hi_idx: Int = (packed_int >> 4) & 0x0F

    # NF4 lookup from shared memory
    var v0_norm: F32 = shared_nf4[lo_idx]
    var v1_norm: F32 = shared_nf4[hi_idx]

    # Column index for weight 0 (col0 = byte_idx * 2)
    var col0 = byte_idx << 1  # byte_idx * 2

    # Compute block/group indices using bit shifts
    var block_id = col0 >> BLOCKSIZE_W_SHIFT  # col0 // 64
    var absmax_idx = row * blocks_per_row + block_id

    # First-level dequant: absmax_q -> code2 lookup
    var q_scale: U8 = absmax_q_ptr[absmax_idx]
    var code_val: F32 = code2_ptr[Int(q_scale)]

    # Second-level dequant: group scale
    var group_id = block_id >> STATE2_SHIFT  # block_id // 256
    var group_idx = row * groups_per_row + group_id
    var scale_base: F32 = scale2_ptr[group_idx]

    # Final scale = code2[absmax_q] * scale2 + offset
    var scale: F32 = fma(code_val, scale_base, offset)

    # Dequantized values
    var v0: F32 = v0_norm * scale
    var v1: F32 = v1_norm * scale

    # Convert to fp16
    var h0: F16 = F16(v0)
    var h1: F16 = F16(v1)

    # Bitcast to u16
    var b0: U16 = bitcast[u16_dtype](h0)
    var b1: U16 = bitcast[u16_dtype](h1)

    # Pack into 32-bit word: [h1:h0] (h0 in low 16 bits, h1 in high 16 bits)
    var packed_out: U32 = (U32(b1) << 16) | U32(b0)

    # Output word index (one 32-bit word per byte = 2 weights)
    # word_stride = n_bytes_per_row (since each byte -> one 32-bit word)
    var word_idx = row * n_bytes_per_row + byte_idx

    out_ptr[word_idx] = packed_out


# -----------------------------
# CPU reference (same logic, stores as fp16 pairs)
# -----------------------------


fn nf4_dequant_cpu_packed(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[
        F16, MutAnyOrigin
    ],  # CPU uses F16 for easier verification
    n_rows: Int,
    n_cols: Int,
):
    var n_bytes_per_row = n_cols >> 1
    var blocks_per_row = (n_cols + BLOCKSIZE_W - 1) >> BLOCKSIZE_W_SHIFT
    var groups_per_row = (blocks_per_row + STATE2_BLOCKSIZE - 1) >> STATE2_SHIFT

    for row in range(n_rows):
        var row_packed_base = row * n_bytes_per_row
        var row_out_base = row * n_cols
        var row_absmax_base = row * blocks_per_row
        var row_group_base = row * groups_per_row

        for byte_idx in range(n_bytes_per_row):
            var packed_byte: U8 = packed[row_packed_base + byte_idx]
            var packed_int: Int = Int(packed_byte)

            var lo_idx: Int = packed_int & 0x0F
            var hi_idx: Int = (packed_int >> 4) & 0x0F

            var v0_norm: F32 = NF4_TABLE[lo_idx]
            var v1_norm: F32 = NF4_TABLE[hi_idx]

            var col0 = byte_idx << 1
            var block_id = col0 >> BLOCKSIZE_W_SHIFT
            var absmax_idx = row_absmax_base + block_id

            var q_scale: U8 = absmax_q[absmax_idx]
            var code_val: F32 = code2[Int(q_scale)]

            var group_id = block_id >> STATE2_SHIFT
            var group_idx = row_group_base + group_id
            var scale_base: F32 = scale2[group_idx]

            var scale: F32 = fma(code_val, scale_base, offset)

            var v0: F32 = v0_norm * scale
            var v1: F32 = v1_norm * scale

            output[row_out_base + col0] = F16(v0)
            output[row_out_base + col0 + 1] = F16(v1)


# -----------------------------
# Correctness check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    # Use a small 2D test case: 4 rows × 128 cols
    var n_rows: Int = 4
    var n_cols: Int = 128
    var n_weights: Int = n_rows * n_cols
    var n_bytes: Int = n_weights // 2
    var n_bytes_per_row: Int = n_cols // 2

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    nf4_dequant_cpu_packed(
        packed_host.unsafe_ptr(),
        absmax_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        scale2_host.unsafe_ptr(),
        F32(offset),
        out_cpu_host.unsafe_ptr(),
        n_rows,
        n_cols,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    # Output buffer: n_bytes 32-bit words (each = 2 fp16)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # 2D grid: grid_x = rows/TILE_ROWS, grid_y = bytes_per_row/TILE_BYTES_X
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    ctx.enqueue_function_checked[
        nf4_dequant_kernel_packed, nf4_dequant_kernel_packed
    ](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_rows,
        n_bytes_per_row,
        blocks_per_row,
        groups_per_row,
        grid_dim=(grid_x, grid_y),
        block_dim=(THREADS_X, THREADS_Y),
    )
    ctx.synchronize()

    # Compare: unpack 32-bit GPU output to fp16 pairs
    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_bytes):
            var packed_word: U32 = out_gpu_host[i]
            var b0: U16 = U16(packed_word & 0xFFFF)
            var b1: U16 = U16((packed_word >> 16) & 0xFFFF)
            var gpu_v0: F16 = bitcast[f16_dtype](b0)
            var gpu_v1: F16 = bitcast[f16_dtype](b1)

            var idx0 = i * 2
            var idx1 = idx0 + 1

            var cpu_v0: F32 = F32(out_cpu_host[idx0])
            var cpu_v1: F32 = F32(out_cpu_host[idx1])
            var gpu_f0: F32 = F32(gpu_v0)
            var gpu_f1: F32 = F32(gpu_v1)

            var diff0: F32 = cpu_v0 - gpu_f0
            var diff1: F32 = cpu_v1 - gpu_f1

            if diff0 < 0.0:
                diff0 = -diff0
            if diff1 < 0.0:
                diff1 = -diff1

            if diff0 > max_abs_diff:
                max_abs_diff = diff0
            if diff1 > max_abs_diff:
                max_abs_diff = diff1

            if i < 8:
                print(idx0, "\t", cpu_v0, "\t", gpu_f0, "\t", cpu_v0 - gpu_f0)
                print(idx1, "\t", cpu_v1, "\t", gpu_f1, "\t", cpu_v1 - gpu_f1)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# -----------------------------


fn bench_layer_packed(
    ctx: DeviceContext,
    n_rows: Int,
    n_cols: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Benchmark packed 32-bit store kernel.
    Returns: seconds (total time for num_iters iterations)
    """
    var n_weights = n_rows * n_cols
    var n_bytes = n_weights // 2
    var n_bytes_per_row = n_cols // 2

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # 2D grid
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel_packed, nf4_dequant_kernel_packed
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_rows,
            n_bytes_per_row,
            blocks_per_row,
            groups_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Mojo Packed 32-bit Store) T4 GPU")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print(
            "Using packed 32-bit store kernel: ",
            THREADS_X,
            "x",
            THREADS_Y,
            " = 1024 threads/block",
        )
        print("Each thread: 1 byte -> 2 fp16 -> 1 x 32-bit store")
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        print("\nConfig 1: rows=", hd1, ", cols=", m1, ", n_weights=", hd1 * m1)

        var t1_up = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer_packed(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        print("\nConfig 2: rows=", hd2, ", cols=", m2, ", n_weights=", hd2 * m2)

        var t2_up = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer_packed(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        print("\nConfig 3: rows=", hd3, ", cols=", m3, ", n_weights=", hd3 * m3)

        var t3_up = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer_packed(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n✅ SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n❌ Need more optimization to reach 1.15x speedup")


NF4 Dequantization Benchmark (Mojo Packed 32-bit Store) T4 GPU

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.010002136 	 -0.010002136 	 0.0
1 	 -0.0069618225 	 -0.0069618225 	 0.0
2 	 -0.0069618225 	 -0.0069618225 	 0.0
3 	 -0.0052490234 	 -0.0052490234 	 0.0
4 	 -0.0052490234 	 -0.0052490234 	 0.0
5 	 -0.0039482117 	 -0.0039482117 	 0.0
6 	 -0.0039482117 	 -0.0039482117 	 0.0
7 	 -0.0028438568 	 -0.0028438568 	 0.0
8 	 -0.0028438568 	 -0.0028438568 	 0.0
9 	 -0.0018472672 	 -0.0018472672 	 0.0
10 	 -0.0018472672 	 -0.0018472672 	 0.0
11 	 -0.00091028214 	 -0.00091028214 	 0.0
12 	 -0.00091028214 	 -0.00091028214 	 0.0
13 	 0.0 	 0.0 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.0007958412 	 0.0007958412 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.
Using packed 32-bit store kernel:  256 x 4  = 1024 threads/block
Each thread: 1 byte -> 2 fp16 -> 1 x 32-bit store

Device: cuda

Config 1: rows= 2048 , cols= 8192 , n

# Warp per NF4 block **T4** GPU

In [5]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation, bitcast
from math import fma, ceildiv

# -----------------------------
# Types and constants
# -----------------------------

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16
comptime u32_dtype = DType.uint32
comptime u16_dtype = DType.uint16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]
comptime U32 = Scalar[u32_dtype]
comptime U16 = Scalar[u16_dtype]

# Bitsandbytes NF4 codebook
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block = 32 bytes
comptime BYTES_PER_NF4_BLOCK = 32  # 64 weights / 2 = 32 bytes
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale
comptime STATE2_SHIFT = 8  # log2(256) for bit shift

# Warp-based tiling:
# - 1 warp (32 threads) = 1 NF4 block (32 bytes = 64 weights)
# - Each thread handles 1 byte = 2 weights
# - Lane 0 computes scale, broadcasts via shared memory
comptime WARP_SIZE = 32
comptime WARPS_PER_BLOCK = 8  # 8 warps per CUDA block = 256 threads
comptime THREADS_PER_BLOCK = WARP_SIZE * WARPS_PER_BLOCK  # 256

# Each CUDA block handles 8 NF4 blocks (8 warps × 32 bytes = 256 bytes = 512 weights)

# ------------------------------------------------------------------------
# GPU kernel: WARP-PER-NF4-BLOCK with shared scale
#   - 1 warp = 1 NF4 block (32 bytes = 64 weights)
#   - Lane 0 computes scale once, stores to shared_scale[warp_id]
#   - All 32 lanes read scale from shared, unpack, multiply, pack, store
#   - Eliminates 31/32 redundant scale loads/computes per NF4 block
# ------------------------------------------------------------------------


fn nf4_dequant_kernel_warp(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[U32, MutAnyOrigin],
    n_nf4_blocks: Int,  # total NF4 blocks = n_weights / 64
    n_bytes: Int,  # total bytes = n_weights / 2
    groups_per_row: Int,  # for 1D layout: ceildiv(n_absmax, STATE2_BLOCKSIZE)
):
    # Shared memory for NF4 codebook (16 values)
    var shared_nf4 = stack_allocation[
        16, F32, address_space = AddressSpace.SHARED
    ]()
    # Shared memory for per-warp scales (one per warp in this CUDA block)
    var shared_scale = stack_allocation[
        WARPS_PER_BLOCK, F32, address_space = AddressSpace.SHARED
    ]()

    var tid = Int(thread_idx.x)
    var warp_id = tid >> 5  # tid // 32
    var lane_id = tid & 31  # tid % 32

    # First 16 threads load the NF4 codebook
    if tid < 16:
        shared_nf4[tid] = NF4_TABLE[tid]

    barrier()

    # Global NF4 block index for this warp
    var nf4_block_idx = Int(block_idx.x) * WARPS_PER_BLOCK + warp_id

    if nf4_block_idx >= n_nf4_blocks:
        return

    # Lane 0 computes the scale for this NF4 block
    if lane_id == 0:
        # absmax_q for this NF4 block
        var q_scale: U8 = absmax_q_ptr[nf4_block_idx]
        var code_val: F32 = code2_ptr[Int(q_scale)]

        # Group index for second-level scale
        var group_id = nf4_block_idx >> STATE2_SHIFT  # nf4_block_idx // 256
        var scale_base: F32 = scale2_ptr[group_id]

        # Final scale = code2[absmax_q] * scale2 + offset
        var scale: F32 = fma(code_val, scale_base, offset)
        shared_scale[warp_id] = scale

    barrier()

    # All lanes read the scale from shared memory
    var scale: F32 = shared_scale[warp_id]

    # Global byte index for this thread
    var byte_idx = nf4_block_idx * BYTES_PER_NF4_BLOCK + lane_id

    if byte_idx >= n_bytes:
        return

    # Read the packed byte
    var packed_byte: U8 = packed_ptr[byte_idx]
    var packed_int: Int = Int(packed_byte)

    # Extract low and high nibbles
    var lo_idx: Int = packed_int & 0x0F
    var hi_idx: Int = (packed_int >> 4) & 0x0F

    # NF4 lookup from shared memory
    var v0_norm: F32 = shared_nf4[lo_idx]
    var v1_norm: F32 = shared_nf4[hi_idx]

    # Dequantized values (just multiply by scale - no per-thread loads!)
    var v0: F32 = v0_norm * scale
    var v1: F32 = v1_norm * scale

    # Convert to fp16
    var h0: F16 = F16(v0)
    var h1: F16 = F16(v1)

    # Bitcast to u16
    var b0: U16 = bitcast[u16_dtype](h0)
    var b1: U16 = bitcast[u16_dtype](h1)

    # Pack into 32-bit word: [h1:h0]
    var packed_out: U32 = (U32(b1) << 16) | U32(b0)

    # Store (one 32-bit word per byte)
    out_ptr[byte_idx] = packed_out


# -----------------------------
# CPU reference (flat 1D layout)
# -----------------------------


fn nf4_dequant_cpu(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[F16, MutAnyOrigin],
    n_bytes: Int,
):
    for byte_idx in range(n_bytes):
        var packed_byte: U8 = packed[byte_idx]
        var packed_int: Int = Int(packed_byte)

        var lo_idx: Int = packed_int & 0x0F
        var hi_idx: Int = (packed_int >> 4) & 0x0F

        var v0_norm: F32 = NF4_TABLE[lo_idx]
        var v1_norm: F32 = NF4_TABLE[hi_idx]

        # NF4 block index for this byte
        var nf4_block_idx = byte_idx >> 5  # byte_idx // 32

        var q_scale: U8 = absmax_q[nf4_block_idx]
        var code_val: F32 = code2[Int(q_scale)]

        var group_id = nf4_block_idx >> STATE2_SHIFT
        var scale_base: F32 = scale2[group_id]

        var scale: F32 = fma(code_val, scale_base, offset)

        var v0: F32 = v0_norm * scale
        var v1: F32 = v1_norm * scale

        var out_idx = byte_idx << 1  # byte_idx * 2
        output[out_idx] = F16(v0)
        output[out_idx + 1] = F16(v1)


# -----------------------------
# Correctness check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    # Small test: 512 weights = 8 NF4 blocks = 256 bytes
    var n_weights: Int = 512
    var n_bytes: Int = n_weights // 2
    var n_nf4_blocks: Int = n_weights // BLOCKSIZE_W
    var n_scale2: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var groups_per_row: Int = n_scale2  # 1D layout

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_nf4_blocks):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    nf4_dequant_cpu(
        packed_host.unsafe_ptr(),
        absmax_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        scale2_host.unsafe_ptr(),
        F32(offset),
        out_cpu_host.unsafe_ptr(),
        n_bytes,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # Grid: each CUDA block handles WARPS_PER_BLOCK NF4 blocks
    var grid_size: Int = ceildiv(n_nf4_blocks, WARPS_PER_BLOCK)

    ctx.enqueue_function_checked[
        nf4_dequant_kernel_warp, nf4_dequant_kernel_warp
    ](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_nf4_blocks,
        n_bytes,
        groups_per_row,
        grid_dim=grid_size,
        block_dim=THREADS_PER_BLOCK,
    )
    ctx.synchronize()

    # Compare
    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_bytes):
            var packed_word: U32 = out_gpu_host[i]
            var b0: U16 = U16(packed_word & 0xFFFF)
            var b1: U16 = U16((packed_word >> 16) & 0xFFFF)
            var gpu_v0: F16 = bitcast[f16_dtype](b0)
            var gpu_v1: F16 = bitcast[f16_dtype](b1)

            var idx0 = i * 2
            var idx1 = idx0 + 1

            var cpu_v0: F32 = F32(out_cpu_host[idx0])
            var cpu_v1: F32 = F32(out_cpu_host[idx1])
            var gpu_f0: F32 = F32(gpu_v0)
            var gpu_f1: F32 = F32(gpu_v1)

            var diff0: F32 = cpu_v0 - gpu_f0
            var diff1: F32 = cpu_v1 - gpu_f1

            if diff0 < 0.0:
                diff0 = -diff0
            if diff1 < 0.0:
                diff1 = -diff1

            if diff0 > max_abs_diff:
                max_abs_diff = diff0
            if diff1 > max_abs_diff:
                max_abs_diff = diff1

            if i < 8:
                print(idx0, "\t", cpu_v0, "\t", gpu_f0, "\t", cpu_v0 - gpu_f0)
                print(idx1, "\t", cpu_v1, "\t", gpu_f1, "\t", cpu_v1 - gpu_f1)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# -----------------------------


fn bench_layer_warp(
    ctx: DeviceContext,
    n_weights: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Benchmark warp-per-NF4-block kernel.
    Returns: seconds (total time for num_iters iterations)
    """
    var n_bytes = n_weights // 2
    var n_nf4_blocks = n_weights // BLOCKSIZE_W
    var n_scale2 = ceildiv(n_nf4_blocks, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var groups_per_row = n_scale2

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_bytes):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_nf4_blocks):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # Grid size
    var grid_size: Int = ceildiv(n_nf4_blocks, WARPS_PER_BLOCK)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel_warp, nf4_dequant_kernel_warp
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_nf4_blocks,
            n_bytes,
            groups_per_row,
            grid_dim=grid_size,
            block_dim=THREADS_PER_BLOCK,
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Warp-per-NF4-Block) T4")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print("Kernel: 1 warp = 1 NF4 block, lane 0 computes shared scale")
        print(
            "Threads per block:",
            THREADS_PER_BLOCK,
            "(",
            WARPS_PER_BLOCK,
            "warps)",
        )
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        var n_w1: Int = hd1 * m1
        print("\nConfig 1: hd=", hd1, ", m=", m1, ", n_weights=", n_w1)

        var t1_up = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer_warp(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        var n_w2: Int = hd2 * m2
        print("\nConfig 2: hd=", hd2, ", m=", m2, ", n_weights=", n_w2)

        var t2_up = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer_warp(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        var n_w3: Int = hd3 * m3
        print("\nConfig 3: hd=", hd3, ", m=", m3, ", n_weights=", n_w3)

        var t3_up = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer_warp(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n Need more optimization to reach 1.15x speedup")


NF4 Dequantization Benchmark (Warp-per-NF4-Block) T4

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.010002136 	 -0.010002136 	 0.0
1 	 -0.0069618225 	 -0.0069618225 	 0.0
2 	 -0.0069618225 	 -0.0069618225 	 0.0
3 	 -0.0052490234 	 -0.0052490234 	 0.0
4 	 -0.0052490234 	 -0.0052490234 	 0.0
5 	 -0.0039482117 	 -0.0039482117 	 0.0
6 	 -0.0039482117 	 -0.0039482117 	 0.0
7 	 -0.0028438568 	 -0.0028438568 	 0.0
8 	 -0.0028438568 	 -0.0028438568 	 0.0
9 	 -0.0018472672 	 -0.0018472672 	 0.0
10 	 -0.0018472672 	 -0.0018472672 	 0.0
11 	 -0.00091028214 	 -0.00091028214 	 0.0
12 	 -0.00091028214 	 -0.00091028214 	 0.0
13 	 0.0 	 0.0 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.0007958412 	 0.0007958412 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.
Kernel: 1 warp = 1 NF4 block, lane 0 computes shared scale
Threads per block: 256 ( 8 warps)

Device: cuda

Config 1: hd= 2048 , m= 8192 , n_weights= 16777216
  up_proj   :  0.4

# Other Kernel Versions

### Kernel 0 – Naive Flat Grid (Baseline, ~25.3 s)

This is the first working Mojo kernel: a 1D flat grid with 256 threads per block, each thread handling a single NF4 byte (two weights), no shared memory, no tiling. It’s simple and good for validating the NF4 + double-dequant math and the timing harness, but on a T4 it lands around 25.3 seconds for the Unsloth test_dequantize workload. roughly 5× slower than the Triton reference and a  “starting point” to compare the later optimized versions against.

In [12]:
%%mojo
from gpu import block_idx, thread_idx, block_dim
from gpu.host import DeviceContext
from memory import UnsafePointer
from math import fma, ceildiv


#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]

# Bitsandbytes NF4 codebook
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block
comptime PAIRS_PER_BLOCK = BLOCKSIZE_W // 2  # 32 bytes per 64 weights
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale

# Kernel launch config
comptime THREADS_PER_BLOCK = 256  # Flat grid: 256 threads per CUDA block


########################################################
# GPU kernel: FLAT GRID approach                       #
#   - Each thread handles 1 byte (2 weights) globally  #
#   - 256 threads per CUDA block for high occupancy.   #
#   - No shared memory, no barrier                     #
########################################################


fn nf4_dequant_kernel(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[F16, MutAnyOrigin],
    n_pairs: Int,  # total bytes (2 weights per byte)
    n_weights: Int,  # total weights
):
    # Global thread ID = byte index (pair index)
    var tid = Int(block_idx.x) * THREADS_PER_BLOCK + Int(thread_idx.x)
    if tid >= n_pairs:
        return

    # Which NF4 block (64 weights = 32 bytes) does this byte belong to?
    var absmax_idx = tid // PAIRS_PER_BLOCK

    # Dequantize the absmax scale
    var q_abs: U8 = absmax_q_ptr[absmax_idx]
    var code_val: F32 = code2_ptr[Int(q_abs)]

    # Second-level scale index (one per 256 absmax entries)
    var scale_idx = absmax_idx >> 8  # absmax_idx // 256
    var scale_base: F32 = scale2_ptr[scale_idx]
    var scale: F32 = fma(code_val, scale_base, offset)

    # Unpack two nf4 4-bit codes from this byte
    var packed_val: U8 = packed_ptr[tid]
    var packed_int: Int = Int(packed_val)

    var hi_idx: Int = (packed_int >> 4) & 0x0F
    var lo_idx: Int = packed_int & 0x0F

    var v0: F32 = NF4_TABLE[hi_idx] * scale
    var v1: F32 = NF4_TABLE[lo_idx] * scale

    var i0: Int = tid * 2
    var i1: Int = i0 + 1

    if i0 < n_weights:
        out_ptr[i0] = F16(v0)
    if i1 < n_weights:
        out_ptr[i1] = F16(v1)


############################################
# CPU reference - uses same math as kernel #
############################################


fn nf4_dequant_cpu(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[F16, MutAnyOrigin],
    n_pairs: Int,
    n_weights: Int,
):
    # Iterate over all bytes (pairs)
    for tid in range(n_pairs):
        # Which NF4 block does this byte belong to?
        var absmax_idx = tid // PAIRS_PER_BLOCK

        # Dequantize the absmax scale (double quant)
        var q_abs: U8 = absmax_q[absmax_idx]
        var code_val: F32 = code2[Int(q_abs)]

        var scale_idx = absmax_idx >> 8
        var scale_base: F32 = scale2[scale_idx]
        var scale: F32 = fma(code_val, scale_base, offset)

        # Unpack two nf4 4-bit codes
        var packed_val: U8 = packed[tid]
        var packed_int: Int = Int(packed_val)

        var hi_idx: Int = (packed_int >> 4) & 0x0F
        var lo_idx: Int = packed_int & 0x0F

        var v0: F32 = NF4_TABLE[hi_idx] * scale
        var v1: F32 = NF4_TABLE[lo_idx] * scale

        var i0: Int = tid * 2
        var i1: Int = i0 + 1

        if i0 < n_weights:
            output[i0] = F16(v0)
        if i1 < n_weights:
            output[i1] = F16(v1)


#####################
# Correctness Check #
#####################

def run_correctness_test(ctx: DeviceContext):
    var n_weights: Int = 64
    var n_pairs: Int = n_weights // 2
    var n_absmax: Int = ceildiv(n_weights, BLOCKSIZE_W)
    var n_scale2: Int = ceildiv(n_absmax, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_pairs):
        var hi: Int = i % 16
        var lo: Int = (15 - i) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    absmax_host[0] = U8(3)
    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))
    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    var packed_ptr = packed_host.unsafe_ptr()
    var absmax_ptr = absmax_host.unsafe_ptr()
    var code2_ptr = code2_host.unsafe_ptr()
    var scale2_ptr = scale2_host.unsafe_ptr()
    var out_cpu_ptr = out_cpu_host.unsafe_ptr()

    nf4_dequant_cpu(
        packed_ptr,
        absmax_ptr,
        code2_ptr,
        scale2_ptr,
        F32(offset),
        out_cpu_ptr,
        n_pairs,
        n_weights,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # Launch kernel: flat grid, 256 threads/block
    var blocks: Int = ceildiv(n_pairs, THREADS_PER_BLOCK)

    ctx.enqueue_function_checked[nf4_dequant_kernel, nf4_dequant_kernel](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_pairs,
        n_weights,
        grid_dim=blocks,
        block_dim=THREADS_PER_BLOCK,
    )
    ctx.synchronize()

    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_weights):
            var cpu_val: F32 = F32(out_cpu_host[i])
            var gpu_val: F32 = F32(out_gpu_host[i])
            var diff: F32 = cpu_val - gpu_val
            if diff < 0.0:
                if -diff > max_abs_diff:
                    max_abs_diff = -diff
            else:
                if diff > max_abs_diff:
                    max_abs_diff = diff

            if i < 16:
                print(i, "\t", cpu_val, "\t", gpu_val, "\t", diff)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


####################################
# Benchmark a single weight matrix #
####################################


fn bench_layer(
    ctx: DeviceContext,
    n_weights: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Returns: seconds (total time for num_iters iterations)
    """
    var n_pairs = n_weights // 2
    var n_absmax = ceildiv(n_weights, BLOCKSIZE_W)
    var n_scale2 = ceildiv(n_absmax, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    for i in range(n_pairs):
        var hi: Int = i % 16
        var lo: Int = (15 - i) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # Flat grid: 256 threads per block, blocks = ceildiv(n_pairs, 256)
    var blocks: Int = ceildiv(n_pairs, THREADS_PER_BLOCK)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel, nf4_dequant_kernel
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_pairs,
            n_weights,
            grid_dim=blocks,
            block_dim=THREADS_PER_BLOCK,
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


###################################
# Unsloth-like benchmark harness  #
###################################


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Mojo kernel)")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        # How many dequant kernel launches per layer we time.
        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        var n_w1: Int = hd1 * m1
        print("\nConfig 1: hd=", hd1, ", m=", m1, ", n_weights=", n_w1)

        var t1_up = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        var n_w2: Int = hd2 * m2
        print("\nConfig 2: hd=", hd2, ", m=", m2, ", n_weights=", n_w2)

        var t2_up = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        var n_w3: Int = hd3 * m3
        print("\nConfig 3: hd=", hd3, ", m=", m3, ", n_weights=", n_w3)

        var t3_up = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        # Compare to Unsloth Triton reference time
        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n Need more optimization to reach 1.15x speedup")



NF4 Dequantization Benchmark (Mojo kernel)

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.45996094 	 -0.45996094 	 0.0
1 	 0.45996094 	 0.45996094 	 0.0
2 	 -0.3203125 	 -0.3203125 	 0.0
3 	 0.33251953 	 0.33251953 	 0.0
4 	 -0.24157715 	 -0.24157715 	 0.0
5 	 0.25878906 	 0.25878906 	 0.0
6 	 -0.18164062 	 -0.18164062 	 0.0
7 	 0.20275879 	 0.20275879 	 0.0
8 	 -0.13085937 	 -0.13085937 	 0.0
9 	 0.15539551 	 0.15539551 	 0.0
10 	 -0.08502197 	 -0.08502197 	 0.0
11 	 0.113220215 	 0.113220215 	 0.0
12 	 -0.041870117 	 -0.041870117 	 0.0
13 	 0.074035645 	 0.074035645 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.036621094 	 0.036621094 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.

Device: cuda

Config 1: hd= 2048 , m= 8192 , n_weights= 16777216
  up_proj   :  0.52246936  s
  gate_proj :  0.535773193  s
  down_proj :  0.534033386  s
  Config 1 total:  1.5922759389999999  s

Config 2: hd= 1024 , m= 4096 , n_weights

### 32 threads per block, 1 block per NF4 block

Each thread handles 1 byte ⇒ 2 weights.

Uses shared memory + barrier to compute scale once per NF4 block.

**Time** 12.728195903

In [ ]:
%%mojo
from gpu import block_idx, thread_idx, block_dim, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation
from math import fma, ceildiv

#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]

# Hard-coded bitsandbytes NF4 codebook (fp32)
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block
comptime PAIRS_PER_BLOCK = BLOCKSIZE_W // 2  # 32 bytes per 64 weights
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale

# -----------------------------
# GPU kernel:
#   1 block = 1 NF4 block (64 weights)
#   32 threads per block, each = 1 byte (2 weights)
# -----------------------------


fn nf4_dequant_kernel(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[F16, MutAnyOrigin],
    n_pairs: Int,  # total bytes (2 weights per byte)
    n_weights: Int,  # total weights
    n_absmax: Int,  # number of 64-weight blocks
):
    # Each block handles exactly one NF4 block (64 weights)
    var absmax_idx = Int(block_idx.x)
    if absmax_idx >= n_absmax:
        return

    # Thread index inside this NF4 block
    var local_pair = Int(thread_idx.x)
    if local_pair >= PAIRS_PER_BLOCK:
        return

    # Global byte index
    var global_pair = absmax_idx * PAIRS_PER_BLOCK + local_pair
    if global_pair >= n_pairs:
        return

    # Shared scale: computed once per NF4 block
    var shared_scale = stack_allocation[
        1, F32, address_space = AddressSpace.SHARED
    ]()

    if thread_idx.x == 0:
        # First-level absmax code for this block
        var q_abs: U8 = absmax_q_ptr[absmax_idx]
        var code_val: F32 = code2_ptr[Int(q_abs)]

        # Second-level scale index (one per 256 absmax entries)
        var scale_idx: Int = absmax_idx >> 8  # absmax_idx // 256
        var scale_base: F32 = scale2_ptr[scale_idx]

        shared_scale[0] = fma(code_val, scale_base, offset)

    barrier()

    var scale: F32 = shared_scale[0]

    # Unpack two nf4 4-bit codes from this byte
    var packed_val: U8 = packed_ptr[global_pair]
    var packed_int: Int = Int(packed_val)

    var hi_idx: Int = (packed_int >> 4) & 0x0F
    var lo_idx: Int = packed_int & 0x0F

    var v0: F32 = NF4_TABLE[hi_idx] * scale
    var v1: F32 = NF4_TABLE[lo_idx] * scale

    var i0: Int = global_pair * 2
    var i1: Int = i0 + 1

    if i0 < n_weights:
        out_ptr[i0] = F16(v0)
    if i1 < n_weights:
        out_ptr[i1] = F16(v1)


# -----------------------------
# CPU reference (same math)
# -----------------------------


fn nf4_dequant_cpu(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[F16, MutAnyOrigin],
    n_pairs: Int,
    n_weights: Int,
    n_absmax: Int,
):
    for absmax_idx in range(n_absmax):
        var q_abs: U8 = absmax_q[absmax_idx]
        var code_val: F32 = code2[Int(q_abs)]

        var scale_idx = absmax_idx >> 8
        var scale_base: F32 = scale2[scale_idx]
        var scale: F32 = fma(code_val, scale_base, offset)

        var base_pair = absmax_idx * PAIRS_PER_BLOCK

        for local_pair in range(PAIRS_PER_BLOCK):
            var pair_idx = base_pair + local_pair
            if pair_idx >= n_pairs:
                break

            var i0: Int = pair_idx * 2
            if i0 >= n_weights:
                break
            var i1: Int = i0 + 1

            var packed_val: U8 = packed[pair_idx]
            var packed_int: Int = Int(packed_val)

            var hi_idx: Int = (packed_int >> 4) & 0x0F
            var lo_idx: Int = packed_int & 0x0F

            var v0: F32 = NF4_TABLE[hi_idx] * scale
            var v1: F32 = NF4_TABLE[lo_idx] * scale

            output[i0] = F16(v0)
            if i1 < n_weights:
                output[i1] = F16(v1)


# -----------------------------
# Correctness sanity check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    var n_weights: Int = 64
    var n_pairs: Int = n_weights // 2
    var n_absmax: Int = ceildiv(n_weights, BLOCKSIZE_W)
    var n_scale2: Int = ceildiv(n_absmax, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_pairs):
        var hi: Int = i % 16
        var lo: Int = (15 - i) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    absmax_host[0] = U8(3)
    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))
    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    var packed_ptr = packed_host.unsafe_ptr()
    var absmax_ptr = absmax_host.unsafe_ptr()
    var code2_ptr = code2_host.unsafe_ptr()
    var scale2_ptr = scale2_host.unsafe_ptr()
    var out_cpu_ptr = out_cpu_host.unsafe_ptr()

    nf4_dequant_cpu(
        packed_ptr,
        absmax_ptr,
        code2_ptr,
        scale2_ptr,
        F32(offset),
        out_cpu_ptr,
        n_pairs,
        n_weights,
        n_absmax,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # Launch kernel (1 block per NF4 block, 32 threads/block)
    var threads_per_block: Int = PAIRS_PER_BLOCK
    var blocks: Int = n_absmax

    ctx.enqueue_function_checked[nf4_dequant_kernel, nf4_dequant_kernel](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_pairs,
        n_weights,
        n_absmax,
        grid_dim=blocks,
        block_dim=threads_per_block,
    )
    ctx.synchronize()

    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_weights):
            var cpu_val: F32 = F32(out_cpu_host[i])
            var gpu_val: F32 = F32(out_gpu_host[i])
            var diff: F32 = cpu_val - gpu_val
            if diff < 0.0:
                if -diff > max_abs_diff:
                    max_abs_diff = -diff
            else:
                if diff > max_abs_diff:
                    max_abs_diff = diff

            if i < 16:
                print(i, "\t", cpu_val, "\t", gpu_val, "\t", diff)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# (similar to one linear layer)
# -----------------------------


fn bench_layer(
    ctx: DeviceContext,
    n_weights: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Returns: seconds (total time for num_iters iterations)
    """
    var n_pairs = n_weights // 2
    var n_absmax = ceildiv(n_weights, BLOCKSIZE_W)
    var n_scale2 = ceildiv(n_absmax, STATE2_BLOCKSIZE)
    if n_scale2 < 1:
        n_scale2 = 1
    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_pairs):
        var hi: Int = i % 16
        var lo: Int = (15 - i) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    var threads_per_block: Int = PAIRS_PER_BLOCK  # 32
    var blocks: Int = n_absmax  # one block per NF4 block

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel, nf4_dequant_kernel
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_pairs,
            n_weights,
            n_absmax,
            grid_dim=blocks,
            block_dim=threads_per_block,
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Mojo kernel)")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        # How many dequant kernel launches per layer we time.
        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192
        var hd1: Int = 2048
        var m1: Int = 8192
        var n_w1: Int = hd1 * m1
        print("\nConfig 1: hd=", hd1, ", m=", m1, ", n_weights=", n_w1)

        var t1_up = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer(ctx, n_w1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        var n_w2: Int = hd2 * m2
        print("\nConfig 2: hd=", hd2, ", m=", m2, ", n_weights=", n_w2)

        var t2_up = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer(ctx, n_w2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        var n_w3: Int = hd3 * m3
        print("\nConfig 3: hd=", hd3, ", m=", m3, ", n_weights=", n_w3)

        var t3_up = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer(ctx, n_w3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        # Compare to Unsloth Triton reference time
        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n Need more optimization to reach 1.15x speedup")





NF4 Dequantization Benchmark (Mojo kernel)

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.45996094 	 -0.45996094 	 0.0
1 	 0.45996094 	 0.45996094 	 0.0
2 	 -0.3203125 	 -0.3203125 	 0.0
3 	 0.33251953 	 0.33251953 	 0.0
4 	 -0.24157715 	 -0.24157715 	 0.0
5 	 0.25878906 	 0.25878906 	 0.0
6 	 -0.18164062 	 -0.18164062 	 0.0
7 	 0.20275879 	 0.20275879 	 0.0
8 	 -0.13085937 	 -0.13085937 	 0.0
9 	 0.15539551 	 0.15539551 	 0.0
10 	 -0.08502197 	 -0.08502197 	 0.0
11 	 0.113220215 	 0.113220215 	 0.0
12 	 -0.041870117 	 -0.041870117 	 0.0
13 	 0.074035645 	 0.074035645 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.036621094 	 0.036621094 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.

Device: cuda

Config 1: hd= 2048 , m= 8192 , n_weights= 16777216
  up_proj   :  0.541392883  s
  gate_proj :  0.557131774  s
  down_proj :  0.552584167  s
  Config 1 total:  1.651108824  s

Config 2: hd= 1024 , m= 4096 , n_weights= 4194

## Kernel 2D Tiling 16bit Store

In [ ]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation
from math import fma, ceildiv

#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype = DType.float32
comptime f16_dtype = DType.float16

comptime U8 = Scalar[packed_dtype]
comptime F32 = Scalar[f32_dtype]
comptime F16 = Scalar[f16_dtype]

# Hard-coded bitsandbytes NF4 codebook (fp32)
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0,
)

# NF4 layout / block structure
comptime BLOCKSIZE_W = 64  # weights per NF4 block
comptime BLOCKSIZE_W_SHIFT = 6  # log2(64) for bit shift
comptime STATE2_BLOCKSIZE = 256  # absmax entries per state2 scale
comptime STATE2_SHIFT = 8  # log2(256) for bit shift

# 2D tiling for T4 optimization
comptime TILE_COLS = 256  # columns per block (matches THREADS_X)
comptime TILE_ROWS = 4  # rows per block (matches THREADS_Y)
comptime THREADS_X = 256
comptime THREADS_Y = 4
# Total: 1024 threads per block

# -----------------------------
# GPU kernel: 2D TILED approach
#   - Each thread handles 1 output weight
#   - 256×4 = 1024 threads per CUDA block
#   - Shared memory for NF4 codebook (16 floats)
#   - 2D grid: (rows/4) × (cols/256)
# -----------------------------


fn nf4_dequant_kernel_2d(
    packed_ptr: UnsafePointer[U8, MutAnyOrigin],
    absmax_q_ptr: UnsafePointer[U8, MutAnyOrigin],
    code2_ptr: UnsafePointer[F32, MutAnyOrigin],
    scale2_ptr: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    out_ptr: UnsafePointer[F16, MutAnyOrigin],
    n_rows: Int,
    n_cols: Int,
    blocks_per_row: Int,  # precomputed on host
    groups_per_row: Int,  # precomputed on host
):
    # Shared memory for NF4 codebook (16 values)
    var shared_nf4 = stack_allocation[
        16, F32, address_space = AddressSpace.SHARED
    ]()

    # First 16 threads of first row load the NF4 codebook
    var tx = Int(thread_idx.x)
    var ty = Int(thread_idx.y)
    if ty == 0 and tx < 16:
        shared_nf4[tx] = NF4_TABLE[tx]

    barrier()

    # 2D position
    var row = Int(block_idx.x) * TILE_ROWS + ty
    var col = Int(block_idx.y) * TILE_COLS + tx

    if row >= n_rows or col >= n_cols:
        return

    # Strides for 2D layout (row-major packed, row-major output)
    var packed_stride = n_cols // 2  # bytes per row
    var out_stride = n_cols  # weights per row

    # Unpack 4-bit: each byte holds 2 weights
    # Weight at col uses byte at col//2, nibble at (col & 1)
    var byte_idx = col // 2
    var nibble_shift = (
        col & 1
    ) * 4  # 0 for even col (low nibble), 4 for odd (high nibble)

    var packed_byte: U8 = packed_ptr[row * packed_stride + byte_idx]
    var q: Int = (Int(packed_byte) >> nibble_shift) & 0x0F

    # NF4 lookup from shared memory
    var normalized: F32 = shared_nf4[q]

    # Compute block/group indices using bit shifts (faster than divides)
    var block_id = col >> BLOCKSIZE_W_SHIFT  # col // 64
    var absmax_idx = row * blocks_per_row + block_id

    # First-level dequant: absmax_q -> code2 lookup
    var q_scale: U8 = absmax_q_ptr[absmax_idx]
    var code_val: F32 = code2_ptr[Int(q_scale)]

    # Second-level dequant: group scale (using bit shift)
    var group_id = block_id >> STATE2_SHIFT  # block_id // 256
    var group_idx = row * groups_per_row + group_id
    var scale_base: F32 = scale2_ptr[group_idx]

    # Final scale = code2[absmax_q] * scale2 + offset
    var scale: F32 = fma(code_val, scale_base, offset)

    # Final dequantized value
    var value: F32 = normalized * scale

    # Store as fp16
    out_ptr[row * out_stride + col] = F16(value)


# -----------------------------
# CPU reference (same 2D logic)
# -----------------------------


fn nf4_dequant_cpu_2d(
    packed: UnsafePointer[U8, MutAnyOrigin],
    absmax_q: UnsafePointer[U8, MutAnyOrigin],
    code2: UnsafePointer[F32, MutAnyOrigin],
    scale2: UnsafePointer[F32, MutAnyOrigin],
    offset: F32,
    output: UnsafePointer[F16, MutAnyOrigin],
    n_rows: Int,
    n_cols: Int,
):
    # Hoist all invariants out of the loop
    var packed_stride = n_cols >> 1  # n_cols // 2
    var blocks_per_row = (n_cols + BLOCKSIZE_W - 1) >> BLOCKSIZE_W_SHIFT
    var groups_per_row = (blocks_per_row + STATE2_BLOCKSIZE - 1) >> STATE2_SHIFT

    for row in range(n_rows):
        # Hoist row-dependent base offsets
        var row_packed_base = row * packed_stride
        var row_out_base = row * n_cols
        var row_absmax_base = row * blocks_per_row
        var row_group_base = row * groups_per_row

        for col in range(n_cols):
            # Unpack nibble using bit ops
            var byte_idx = col >> 1  # col // 2
            var nibble_shift = (col & 1) << 2  # (col & 1) * 4
            var packed_byte: U8 = packed[row_packed_base + byte_idx]
            var q: Int = (Int(packed_byte) >> nibble_shift) & 0x0F

            # NF4 lookup
            var normalized: F32 = NF4_TABLE[q]

            # Block/group indices using bit shifts
            var block_id = col >> BLOCKSIZE_W_SHIFT  # col // 64
            var absmax_idx = row_absmax_base + block_id

            var q_scale: U8 = absmax_q[absmax_idx]
            var code_val: F32 = code2[Int(q_scale)]

            var group_id = block_id >> STATE2_SHIFT  # block_id // 256
            var group_idx = row_group_base + group_id
            var scale_base: F32 = scale2[group_idx]

            var scale: F32 = fma(code_val, scale_base, offset)
            var value: F32 = normalized * scale

            output[row_out_base + col] = F16(value)


# -----------------------------
# Correctness sanity check
# -----------------------------


def run_correctness_test(ctx: DeviceContext):
    # Use a small 2D test case: 4 rows × 128 cols
    var n_rows: Int = 4
    var n_cols: Int = 128
    var n_weights: Int = n_rows * n_cols
    var n_pairs: Int = n_weights // 2  # total bytes

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)
    var out_cpu_host = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # Fill deterministic pattern
    for i in range(n_pairs):
        # Pack two nibbles: low nibble = i%16, high nibble = (i+1)%16
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # CPU ref
    nf4_dequant_cpu_2d(
        packed_host.unsafe_ptr(),
        absmax_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        scale2_host.unsafe_ptr(),
        F32(offset),
        out_cpu_host.unsafe_ptr(),
        n_rows,
        n_cols,
    )

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)

    # 2D grid: grid_x = rows/TILE_ROWS, grid_y = cols/TILE_COLS
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_cols, TILE_COLS)

    ctx.enqueue_function_checked[nf4_dequant_kernel_2d, nf4_dequant_kernel_2d](
        packed_dev,
        absmax_dev,
        code2_dev,
        scale2_dev,
        F32(offset),
        out_dev,
        n_rows,
        n_cols,
        blocks_per_row,
        groups_per_row,
        grid_dim=(grid_x, grid_y),
        block_dim=(THREADS_X, THREADS_Y),
    )
    ctx.synchronize()

    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0

        print("idx\tCPU\t\tGPU\t\tdiff")
        for i in range(n_weights):
            var cpu_val: F32 = F32(out_cpu_host[i])
            var gpu_val: F32 = F32(out_gpu_host[i])
            var diff: F32 = cpu_val - gpu_val
            if diff < 0.0:
                if -diff > max_abs_diff:
                    max_abs_diff = -diff
            else:
                if diff > max_abs_diff:
                    max_abs_diff = diff

            if i < 16:
                print(i, "\t", cpu_val, "\t", gpu_val, "\t", diff)

        print("max_abs_diff =", max_abs_diff)
        print(
            "Correctness test PASSED!" if max_abs_diff
            < 1e-3 else "Correctness test FAILED!"
        )


# -----------------------------
# Benchmark a single weight matrix
# For benchmark, treat n_weights as a 1D array reshaped to 2D
# We'll use n_rows=hd, n_cols=m to match Unsloth's shapes
# -----------------------------


fn bench_layer_2d(
    ctx: DeviceContext,
    n_rows: Int,
    n_cols: Int,
    num_iters: Int,
) raises -> Float64:
    """
    Benchmark 2D dequant kernel.
    """
    var n_weights = n_rows * n_cols
    var n_pairs = n_weights // 2

    var blocks_per_row = ceildiv(n_cols, BLOCKSIZE_W)
    var n_absmax = n_rows * blocks_per_row
    var groups_per_row = ceildiv(blocks_per_row, STATE2_BLOCKSIZE)
    var n_scale2 = n_rows * groups_per_row
    if n_scale2 < 1:
        n_scale2 = 1

    var offset: F32 = 0.01

    # Host buffers
    var packed_host = ctx.enqueue_create_host_buffer[packed_dtype](n_pairs)
    var absmax_host = ctx.enqueue_create_host_buffer[packed_dtype](n_absmax)
    var code2_host = ctx.enqueue_create_host_buffer[f32_dtype](16)
    var scale2_host = ctx.enqueue_create_host_buffer[f32_dtype](n_scale2)

    ctx.synchronize()

    # Fill patterns
    for i in range(n_pairs):
        var lo: Int = i % 16
        var hi: Int = (i + 1) % 16
        var byte_val: Int = (hi << 4) | lo
        packed_host[i] = U8(byte_val)

    for i in range(n_absmax):
        absmax_host[i] = U8(i % 16)

    for i in range(16):
        code2_host[i] = F32(0.1 * Float64(i))

    for i in range(n_scale2):
        scale2_host[i] = F32(1.5)

    # Device buffers
    var packed_dev = ctx.enqueue_create_buffer[packed_dtype](n_pairs)
    var absmax_dev = ctx.enqueue_create_buffer[packed_dtype](n_absmax)
    var code2_dev = ctx.enqueue_create_buffer[f32_dtype](16)
    var scale2_dev = ctx.enqueue_create_buffer[f32_dtype](n_scale2)
    var out_dev = ctx.enqueue_create_buffer[f16_dtype](n_weights)

    ctx.enqueue_copy(dst_buf=packed_dev, src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_dev, src_buf=absmax_host)
    ctx.enqueue_copy(dst_buf=code2_dev, src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=scale2_dev, src_buf=scale2_host)
    ctx.synchronize()

    # 2D grid
    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_cols, TILE_COLS)

    # Compile once
    var compiled_kernel = ctx.compile_function_checked[
        nf4_dequant_kernel_2d, nf4_dequant_kernel_2d
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled_kernel,
            packed_dev,
            absmax_dev,
            code2_dev,
            scale2_dev,
            F32(offset),
            out_dev,
            n_rows,
            n_cols,
            blocks_per_row,
            groups_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    # Warmup
    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    var seconds: Float64 = Float64(total_ns) / 1e9

    return seconds


# -----------------------------
# Unsloth-like benchmark harness
# -----------------------------


def main():
    with DeviceContext() as ctx:
        print("=" * 60)
        print("NF4 Dequantization Benchmark (Mojo 2D Tiled Kernel)")
        print("=" * 60)

        print("\n--- Correctness Test ---")
        run_correctness_test(ctx)

        # Dequant kernel launches per layer
        var NUM_ITERS_PER_LAYER: Int = 1000

        print("\n--- Unsloth-style Dequant Bench ---")
        print("Each layer runs", NUM_ITERS_PER_LAYER, "kernel launches.")
        print(
            "Using 2D tiled kernel: ",
            THREADS_X,
            "x",
            THREADS_Y,
            " = 1024 threads/block",
        )
        print("\nDevice:", ctx.api())

        var total_time: Float64 = 0.0

        # Config 1: hd=2048, m=8192 (treat as 2048 rows × 8192 cols)
        var hd1: Int = 2048
        var m1: Int = 8192
        print("\nConfig 1: rows=", hd1, ", cols=", m1, ", n_weights=", hd1 * m1)

        var t1_up = bench_layer_2d(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t1_up, " s")
        var t1_gate = bench_layer_2d(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t1_gate, " s")
        var t1_down = bench_layer_2d(ctx, hd1, m1, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t1_down, " s")

        var config1_time = t1_up + t1_gate + t1_down
        print("  Config 1 total: ", config1_time, " s")
        total_time += config1_time

        # Config 2: hd=1024, m=4096
        var hd2: Int = 1024
        var m2: Int = 4096
        print("\nConfig 2: rows=", hd2, ", cols=", m2, ", n_weights=", hd2 * m2)

        var t2_up = bench_layer_2d(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t2_up, " s")
        var t2_gate = bench_layer_2d(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t2_gate, " s")
        var t2_down = bench_layer_2d(ctx, hd2, m2, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t2_down, " s")

        var config2_time = t2_up + t2_gate + t2_down
        print("  Config 2 total: ", config2_time, " s")
        total_time += config2_time

        # Config 3: hd=4096, m=14336
        var hd3: Int = 4096
        var m3: Int = 14336
        print("\nConfig 3: rows=", hd3, ", cols=", m3, ", n_weights=", hd3 * m3)

        var t3_up = bench_layer_2d(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  up_proj   : ", t3_up, " s")
        var t3_gate = bench_layer_2d(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  gate_proj : ", t3_gate, " s")
        var t3_down = bench_layer_2d(ctx, hd3, m3, NUM_ITERS_PER_LAYER)
        print("  down_proj : ", t3_down, " s")

        var config3_time = t3_up + t3_gate + t3_down
        print("  Config 3 total: ", config3_time, " s")
        total_time += config3_time

        print("\n" + "=" * 60)
        print("TOTAL Mojo NF4 dequant time:", total_time, "seconds")
        print("=" * 60)

        # Compare to Unsloth Triton reference time
        var unsloth_time: Float64 = 5.320246934890747
        var speedup = unsloth_time / total_time
        print("\nUnsloth Triton time (ref): ", unsloth_time, " s")
        print("Mojo kernel time:          ", total_time, " s")
        print("Speedup (Triton / Mojo):   ", speedup, "x")

        if speedup >= 1.15:
            print("\n✅ SUCCESS: Achieved >= 1.15x speedup!")
        else:
            print("\n❌ Need more optimization to reach 1.15x speedup")



NF4 Dequantization Benchmark (Mojo 2D Tiled Kernel)

--- Correctness Test ---
idx	CPU		GPU		diff
0 	 -0.010002136 	 -0.010002136 	 0.0
1 	 -0.0069618225 	 -0.0069618225 	 0.0
2 	 -0.0069618225 	 -0.0069618225 	 0.0
3 	 -0.0052490234 	 -0.0052490234 	 0.0
4 	 -0.0052490234 	 -0.0052490234 	 0.0
5 	 -0.0039482117 	 -0.0039482117 	 0.0
6 	 -0.0039482117 	 -0.0039482117 	 0.0
7 	 -0.0028438568 	 -0.0028438568 	 0.0
8 	 -0.0028438568 	 -0.0028438568 	 0.0
9 	 -0.0018472672 	 -0.0018472672 	 0.0
10 	 -0.0018472672 	 -0.0018472672 	 0.0
11 	 -0.00091028214 	 -0.00091028214 	 0.0
12 	 -0.00091028214 	 -0.00091028214 	 0.0
13 	 0.0 	 0.0 	 0.0
14 	 0.0 	 0.0 	 0.0
15 	 0.0007958412 	 0.0007958412 	 0.0
max_abs_diff = 0.0
Correctness test PASSED!

--- Unsloth-style Dequant Bench ---
Each layer runs 1000 kernel launches.
Using 2D tiled kernel:  256 x 4  = 1024 threads/block

Device: cuda

Config 1: rows= 2048 , cols= 8192 , n_weights= 16777216
  up_proj   :  0.243986434  s
  gate_proj :  0.250721

# Rest of the Unsloth Puzzles

## B) Make `QLoRA` work with `FSDP2` [Difficulty: Medium to Hard] [Max points: 10]

<a name="FSDP2"></a>

1. Goal: Write a single Python script to finetune Llama 3.1 8B on 2x or more GPUs with FSDP2.

2. You must showcase this working in a free **Kaggle notebook with 2 x Tesla T4 GPUs**.

3. Pipeline parallelism is also fine, but must utilize [`zero bubble scheduling`](https://pytorch.org/docs/stable/distributed.pipelining.html#torch.distributed.pipelining.schedules.ScheduleInterleavedZeroBubble) somehow.

4. Can use a pre-quantized 4bit BnB safetensor file from [Unsloth's HF page](https://huggingface.co/unsloth) or a full 16bit one, but must do QLoRA.

5. Can use `accelerate` but must be FSDP2 or related - you can investigate https://github.com/huggingface/accelerate/pull/3394, Torch Titan, other repos etc.

6. Must be fully `transformers` compatible - so we must use `TrainingArguments` and `Trainer`, or `TRL` related classes.

7. The loss must be equivalent to single GPU training.

8. You must enable all features in FSDP2 - ie showcase offloading, checkpointing, mixed precision training etc.

9. You can use `nf4` from `torch AO`, but best from `bitsandbytes`.

10. Finally showcase everything working in a free Kaggle 2x Tesla T4 notebook.

In [ ]:
# HELPFUL functions to undo Unsloth patches:
import sys

def remove_patched_module(package_name):
    modules_to_delete = [
        name for name in sys.modules
        if name == package_name or name.startswith(package_name + ".")
    ]
    for name in modules_to_delete: del sys.modules[name]

remove_patched_module("trl")
remove_patched_module("transformers")
remove_patched_module("peft")
remove_patched_module("bitsandbytes")

Below is an example script which should run fine in Kaggle 2x Telsa T4s:

In [ ]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,"\
    "roundup_power2_divisions:[32:256,64:128,256:64,>:32]"

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

max_seq_length = 2048
torch.set_default_dtype(torch.float16)
model_name = "unsloth/meta-Llama-3.1-8B-Instruct-bnb-4bit"
dtype = torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_4bit              = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_compute_dtype    = dtype,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    attn_implementation = "sdpa",
    quantization_config = bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"

lora_config = LoraConfig(
    r = 64,
    lora_alpha = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    task_type = TaskType.CAUSAL_LM,
)

# Get LoRA and setup model
model = get_peft_model(model, lora_config)
with torch.no_grad():
    for name, param in model.named_parameters():
        if ".lora_A." in name or ".lora_B." in name: param.requires_grad_(True)
        else: param.requires_grad_(False)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

# Get dataset
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train[:10%]")

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

unified_chip2.jsonl:   0%|          | 0.00/95.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Reminder your code must have the same loss curve over 60 steps or so.

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    processing_class = tokenizer,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 1,
        max_steps = 10,
        logging_steps = 1,
        output_dir = "outputs",
        seed = 3407,
        max_seq_length = max_seq_length,
        fp16 = model.get_input_embeddings().weight.dtype == torch.float16,
        bf16 = model.get_input_embeddings().weight.dtype == torch.bfloat16,
        report_to = "none", # For W&B
        dataset_num_proc = 4,
    ),
)
trainer.train()

Applying chat template to train dataset (num_proc=4):   0%|          | 0/21029 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=4):   0%|          | 0/21029 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=4):   0%|          | 0/21029 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,2.039400
2,2.254400
3,2.649500
4,1.936600
5,1.886300
6,1.873100
7,1.577100
8,1.687100
9,1.540900
10,1.792700


TrainOutput(global_step=10, training_loss=1.9237143635749816, metrics={'train_runtime': 91.7565, 'train_samples_per_second': 0.872, 'train_steps_per_second': 0.109, 'total_flos': 461650822987776.0, 'train_loss': 1.9237143635749816})

In [ ]:
del model
import gc
gc.collect()
torch.cuda.empty_cache()

## Marking Criteria for B) Max points = 10
```python
if attemped_B:
    B_score = 0
    if FSDP2_works_with_QLoRA:
        if torch_compile_works: B_score += 5
        else: B_score += 3
        if uses_part_A_and_single_kernel_and_faster: B_score += 3
        elif uses_torchAO:
            if torchAO_slower_than_BnB: B_score -= 3
    elif TP_or_PP_with_QLoRA:
        if zero_bubble: B_score += 3
        else: B_score += 2
    elif FSDP1_works_with_QLoRA:
        B_score += 1
    if kaggle_notebook_2_tesla_t4_example:
        B_score += 2
    else:
        B_score = 0
    final_score += B_score
else:
    final_score -= 2
```

## C) Make `torch.compile` work without graph breaks for QLoRA [Difficulty: Easy to Medium] [Max points: 9]

<a name="COMPILE"></a>

1. Goal: Write a single Python script like task B), except the goal is to `torch.compile` all modules if possible.

2. There must NOT be graph breaks, and excessive re-compilations should not be seen.

3. You should have say max 30 compilations. Over 60 is definitely wrong.

4. The loss must match with the non compiled module.

5. Utilize patching as much as possible.

6. Think about which areas might need disabling for compilation. Think about regional compilation. How do we compile sections efficiently?

7. Log memory / VRAM usage, and monitor speedups as well.

8. Must work for QLoRA.

We provided a script below, and showcased how to detect if graph breaks are seen. We also torch compiled the MLP for Llama:

In [ ]:
import torch
torch_compile_options = torch_compile_options = {
    "epilogue_fusion"   : True,
    "max_autotune"      : True,
    "shape_padding"     : True,
    "trace.enabled"     : True,
    "triton.cudagraphs" : False,
}

@torch.compile(fullgraph = False, dynamic = True, options = torch_compile_options)
def compiled_llama_mlp(self, x):
    down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
    return down_proj

import transformers.models.llama.modeling_llama
transformers.models.llama.modeling_llama.LlamaMLP.forward = compiled_llama_mlp

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = \
    "expandable_segments:True,"\
    "roundup_power2_divisions:[32:256,64:128,256:64,>:32]"

max_seq_length = 1024
torch.set_default_dtype(torch.float16)
model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
dtype = torch.float16
bnb_config = BitsAndBytesConfig(
    load_in_4bit              = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type       = "nf4",
    bnb_4bit_compute_dtype    = dtype,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    attn_implementation = "sdpa",
    quantization_config = bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"

lora_config = LoraConfig(
    r = 32,
    lora_alpha = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_dropout = 0,
    bias = "none",
    task_type = TaskType.CAUSAL_LM,
)

# Get LoRA and setup model
model = get_peft_model(model, lora_config)
with torch.no_grad():
    for name, param in model.named_parameters():
        if ".lora_A." in name or ".lora_B." in name: param.requires_grad_(True)
        else: param.requires_grad_(False)

# Currently GC will cause torch.compile to be disabled, so disable it
# model.gradient_checkpointing_enable()
model.enable_input_require_grads()

# Get dataset
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train[:10%]")

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We provide full logging for `torch.compile` like below:

In [ ]:
# Must show all graph breaks are not seen with torch.compile
import os
os.environ["TORCHDYNAMO_VERBOSE"] = "1"
os.environ["TORCHINDUCTOR_FORCE_DISABLE_CACHES"] = "1"
os.environ["TORCHINDUCTOR_COMPILE_THREADS"] = "1"

import logging
torch._inductor.config.debug = True
torch._logging.set_logs(
    dynamo = logging.WARN,
    inductor = logging.WARN,
    graph_breaks = True,
    recompiles = True,
    recompiles_verbose = True,
    compiled_autograd_verbose = True,
    # aot_joint_graph = True, # Enable for more logs
    # aot_graphs = True,
)
torch._dynamo.config.verbose = True
torch._dynamo.config.suppress_errors = False

When we execute the code below, we can see graph breaks - remove them.

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    processing_class = tokenizer,
    args = SFTConfig(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 1,
        max_steps = 10,
        logging_steps = 1,
        output_dir = "outputs",
        seed = 3407,
        max_seq_length = max_seq_length,
        fp16 = model.get_input_embeddings().weight.dtype == torch.float16,
        bf16 = model.get_input_embeddings().weight.dtype == torch.bfloat16,
        report_to = "none", # For W&B
        dataset_num_proc = 4,
    ),
)
trainer.train()

Log all your steps for debugging in a Colab (maybe this one). Edward's blog http://blog.ezyang.com/, Horace's blogs https://www.thonking.ai/, Slaying OOMs by Jane & Mark: ttps://www.youtube.com/watch?v=UvRl4ansfCg could be useful.

## Marking Criteria for C) Max points = 9
```python
if attemped_C:
    C_score = 0
    if uses_flex_attention:
        if dynamic_sequence_length_works: C_score += 3
        else: C_score += 1
    if no_torch_compile_BnB: C_score -= 2
    elif use_part_A: C_score += 1
    elif torch_compile_BnB: C_score += 1

    if attention_compiled:
        if excessive_recompilation: C_score -= 3
        else: C_score += 2
    if mlp_compiled:
        if excessive_recompilation: C_score -= 3
        C_score += 1

    if not loss_compiled: C_score -= 1
    if not layernorms_compiled: C_score -= 3

    if max_autotune_triton_matmul:
        if excessive_recompilation: C_score -= 2
        else: C_score += 2
    
    final_score += C_score
else:
    final_score -= 1
```

## D) Help solve 🦥 Unsloth issues! [Difficulty: Varies] [Max points: 12]

<a name="ISSUES"></a>

Head over to https://github.com/unslothai/unsloth, and find some issues which are still left standing / not resolved. The tag **currently fixing** might be useful.

Each successfully accepted and solved issue will also have \$100 to \$1000 of bounties.

It's best to attempt these features:

* **<ins>Tool Calling</ins>** [Points = 1] Provide a tool calling Colab notebook and make it work inside of Unsloth. <ins>Bounty: \$1000</ins>

* **<ins>GGUF Vision support</ins>** [Points = 1] Allow exporting vision finetunes to GGUF directly. Llava and Qwen VL must work. <ins>Bounty: \$500</ins>

* **<ins>Refactor Attention</ins>** [Points = 2] Refactor and merge xformers, SDPA, flash-attn, flex-attention into a simpler interface. Must work seamlessly inside of Unsloth. <ins>Bounty: \$350</ins>

* <font color='red'>DONE</font> ** <ins><del>Windows support</del></ins>** [Points = 2] Allow `pip install unsloth` to work in Windows - Triton, Xformers, bitsandbytes should all function. You might need to edit `pyproject.toml`. Confirm it works. <ins>Bounty: \$300</ins>

* **<ins>Support Sequence Classification</ins>** [Points = 1] Create patching functions to patch over AutoModelForSequenceClassification, and allow finetuner to use AutoModelForSequenceClassification. <ins>Bounty: \$200</ins>

* **<ins>VLMs Data Collator</ins>** [Points = 1] Make text & image mixing work efficiently -so some inputs can be text only. Must work on Qwen, Llama, Pixtral. <ins>Bounty: \$100</ins>

* <font color='red'>DONE</font> **<ins>VLMs image resizing</ins>** [Points = 1] Allow finetuner to specify maximum image size, or get it from the config.json file. Resize all images to specific size to reduce VRAM. <ins>Bounty: \$100</ins>

* **<ins>Support Flex Attention</ins>** [Points = 2] Allow dynamic sequence lengths without excessive recompilation. Make this work on SWAs and normal causal masks. Also packed sequence masks. <ins>Bounty: \$100</ins>

* <font color='red'>DONE</font> **<ins>VLMs train only on completions</ins>** [Points = 1] Edit `train_on_responses_only` to allow it to work on VLMs. <ins>Bounty: \$100</ins>


## Marking Criteria for D) Max points = 12
```python
if attemped_D:
    D_score = 0
    for subtask in subtasks:
        if sucessfully_completed_subtask:
            D_score += score_for_subtask
    final_score += D_score
```

## E) Memory Efficient Backprop [Difficulty: Medium to Hard] [Max points: 10]

<a name="MATH"></a>

In LLMs, the last layer is a projection matrix to calculate the probabilities of the next token, ie $\sigma(XW)$. However, if the vocabulary size is very large, say 128K, then the materialization of the logits causes VRAM spikes.

For example, if the `bsz = 4, qlen = 4096, hd = 4096, vocab = 128K`, then the memory usage for the logits in bfloat16 would be 4GB. In the worst case, we might even need to upcast logits to float32, so 8GB is needed.

In Unsloth, we utilize [Apple's Cut Cross Entropy Loss](https://machinelearning.apple.com/research/cut-your-losses) to reduce VRAM usage, by allowing a Triton kernel to create the logits on the fly to calculate the cross entropy loss. But this does not generalize well to other functions.

Our goal is to generalize this ultimately, but directly creating logits on the fly will be hard. Instead, let's take a slightly less complex approach. Let's first review some stuff. We first notice that during the normal case after forming the intermediate logits for 2 batches, we then do a gather function to aggregate the intermediate results into a single column:
$$
\begin{align}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \times W &= \begin{bmatrix} x_1 W \\ x_2 W \end{bmatrix} \\
f \bigg( \begin{bmatrix} x_1 W \\ x_2 W \end{bmatrix} \bigg) &= \begin{pmatrix} y_1 \\ y_2 \end{pmatrix}
\end{align}
$$

So, if we can somehow skip the materialization of the intermediate logits, and just output the output of `f`, we can save a lot of VRAM!

Notice during backpropagation we can use the chain rule:
$$
\begin{align}
\frac{dL}{dX} &= \frac{dL}{dy} \frac{dy}{dX} ; \frac{dL}{dW} = \frac{dL}{dy} \frac{dy}{dW} \\
\frac{dL}{dy} &= \text{Downstream from backprop} \\
\frac{dy}{dX} &= W^T \\
\frac{dy}{dW} &= X^T \\
\frac{dL}{dX} &= \frac{dL}{dy} W^T \\
\frac{dL}{dW} &= X^T \frac{dL}{dy} \\
\end{align}
$$

If we simply compute the intermediate tensors on the fly via batches, say we do batch 1, then batch 2, we can reduce VRAM usage from 4GB to 2GB!

$$
\begin{align}
\frac{dL}{dX} &= \begin{bmatrix} \frac{dL_1}{dy_1} W^T \\ \frac{dL_2}{dy_2} W^T \end{bmatrix} \\
\frac{dL}{dW} &= \bigg( X_1^T \frac{dL_1}{dy_1} + X_2^T  \frac{dL_2}{dy_2} \bigg)
\end{align}
$$

1. Your goal is to write a `torch.autograd.Function` with a `forward` and `backward` pass showcasing this memory efficient implementation.

2. You must NOT hard code the derivatives - move the transformation function from the logits / intermeditate tensors to a smaller tensor as a separate function which can allow `autograd` to pass through it.

3. As a hint, look at `torch.checkpoint` at https://github.com/pytorch/pytorch/blob/main/torch/utils/checkpoint.py. Also, don't forget about the upstream gradients! We need to multiply them to the current gradients!

4. Make the Cross Entropy Loss work. You must show other functions working as well.

In [ ]:
def transformation_function(batch, linear, labels):
    x = linear(batch).float() # Up projection to large space
    from torch.nn import CrossEntropyLoss
    down_projection_function = CrossEntropyLoss(reduction = "mean")
    # Down projection to small space
    loss = down_projection_function(x.view(-1, x.shape[-1]), labels.view(-1))
    return loss

class MemoryEfficientLinear(torch.autograd.Function):
    @staticmethod
    def forward(ctx, X, linear, labels, forward_function):
        outputs = []
        # EDIT THIS FUNCTION
        output = forward_function(X, linear, labels)
        ctx.save_for_backward(X)
        return output

    @staticmethod
    def backward(ctx, dY):
        X = ctx.saved_tensors
        # EDIT THIS FUNCTION
        return X, None, None, None

To test your implementation, it should not OOM for large inputs. Also, check the gradient is actually equivalent via `torch.allclose` in the normal approach.

## Marking Criteria for E) Max points = 10
```python
if attemped_E:
    E_score = 0
    if VRAM_50_percent_reduction: E_score += 2
    if remove_float32_upcast: E_score = 0
    if show_ce_loss_works: E_score += 1
    if show_other_functions_work: E_score += 1
    if hardcoded_gradients: E_score = 0
    if allows_dynamic_chunk_sizes: E_score += 1
    if llama_1B_training_loss_matches: E_score += 1
    else: E_score = 0
    if GRPO_memory_efficient_linear_works: E_score += 4
    final_score += E_score
else:
    final_score += 0
```

# Submission Steps

<a name="SUBMISSION"></a>

1. All code should be in a public Github (Apache 2 Licensed)
2. Kaggle notebooks and Colab notebooks should be linked in the README, and can be accessible through Colab / Kaggle.
3. If attaching notebooks, must attach fully run ones - do not just add a notebook without running it. Kaggle notebook must be public, and run.
4. Submit the Github to https://forms.gle/crSYnsGq3t1ck5TB9 If you want to send a private repo, please add me as a Github collaborate @danielhanchen
5. Provide screenshots, graphs, plots, etc especially for training loss curves.
6. We will comment and respond inside your Github repo. There will get 1 interview as well as a final step!

### Clarifications:
1. We'll compensate you if we interview you but don't hire you
2. \$100-\$1000 bounties for Task 4
3. Submissions must be Apache-2 licensed
4. Task 4 involves solving Github issues for OSS Unsloth
5. No time limit: rolling basis
6. US based preferred